In [ ]:
; If using Jupyter Lab, run
;   jupyter labextension install @jupyterlab/javascript-extension
; before using this notebook.
(ns problemset
  (:refer-clojure :exclude [get contains? keys empty? dissoc assoc get-in
                            map replicate apply])
  (:require [clojure.pprint :refer [pprint]]
            [clojure.repl :refer [doc source]]
            [clojure.string :refer [index-of]]
            [metaprob.syntax :refer :all]
            [metaprob.compound :refer :all] ;; get, contains?, keys, empty?, dissoc, assoc, get-in
            [metaprob.builtin :refer :all] ;; map, reduce, replicate, apply
            [metaprob.trace :refer :all]
            [metaprob.autotrace :refer :all]
            [metaprob.prelude :refer :all]
            [metaprob.inference :refer :all]
            [metaprob.distributions :refer :all]
            [metaprob.tutorial.jupyter :refer :all]))

;; After running this cell, you should see the message
;; "Inline visualization functions have been enabled."
(enable-inline-viz)

# 6.885 Problem Set 2: PPLs, Behind the Scenes
In your last problem set, you learned to use [Gen](https://github.com/probcomp/Gen), a state-of-the-art probabilistic programming system. The focus was on _modeling_ generative processes with probabilistic programs, and writing good _inference programs_ using Gen's standard library. [Metaprob](https://github.com/probcomp/Metaprob), the language we'll use in this problem set, can be used in much the same way, as a tool for modeling and inference. But that won't be the focus of this problem set. Instead, we'll go deeper: **we'll examine a _minimal set of features_ for a probabilistic programming language, and explore how those features can be combined to implement a _standard inference library._**

<p style="color:blue"> Please read and do the problems in  ALL of the first four sections. Some exercises depend on earlier ones, so please attempt the problems in order.</p>

1. **[Why do we need a language?](#1)**

    We introduce three modeling and inference problems, and explore how far we can go _without_ a probabilistic programming language.
    
    _Subsections_: [1.1: Three example models](#11); [1.2: Simulation-based inference](#12); [1.3: Summary](#13)
        
    _Problems_: [Problem 1.2.1](#121), [Problem 1.2.2](#122), [Problem 1.2.3](#123), [Problem 1.2.4](#124)
    
    _Inference algorithms_: forward simulation, rejection sampling


2. **[Generative functions in Metaprob: intuition](#2)**

    A key abstraction in probabilistic programming is the _generative function_, a stochastic simulator augmented with additional probabilistic capabilities. We introduce Metaprob's generative function interface, and use it to implement smarter inference algorithms.
    
    _Subsections_: [2.1: Tracing and scoring](#21); [2.2: Primitive generative functions](#22); [2.3: Compound generative functions](#23); [2.4: Score-based inference with GFs](#24)
        
    _Problems_: [Problem 2.1.1](#221), [Problem 2.1.2](#222), [Problem 2.1.3](#223), [Problem 2.3.1](#231), [Problem 2.3.2](#232), [Problem 2.3.3](#233), [Problem 2.4.1](#241), [Problem 2.4.2](#242), [Problem 2.4.3](#243)

    _Inference algorithms_: importance resampling, likelihood weighting, importance sampling


3. **[A library for MCMC](#3)**

    On top of Metaprob's generative function interface, we can write a small inference programming library for MCMC (much like Gen's).
    
    _Subsections_: [3.1: MH with symmetric proposals](#31); [3.2: MH with asymmetric proposals](#32); [3.3: Gibbs sampling](#33)
    
    _Problems_: [Problem 3.1.1](#311), [Problem 3.1.2](#312), [Problem 3.1.3](#313), [Problem 3.2.1](#321), [Problem 3.2.2](#322), [Problem 3.2.3](#323), [Problem 3.3.1](#331)
    
    _Inference algorithms_: Gaussian drift MH, Gibbs sampling, custom proposal MH
    


4. **[Generative functions in Metaprob: the full picture](#4)**

    We develop a more complete understanding of Metaprob's generative function interface, and introduce the idea of a _custom generative function_: a generative function with compositional custom inference machinery built in. We see that our standard inference library automatically respects (and benefits from) this customization.
    
    _Subsections_: [4.1: Importance sampling with a custom proposal](#41); [4.2: The `make-constrained-generator` method](#42); [4.3: Custom generative functions](#43); [4.4: Resimulation MH proposals](#44)
    
    _Problems_: [Problem 4.1.1](#411), [Problem 4.1.2](#412), [Problem 4.1.3](#413), [Problem 4.3.1](#431), [Problem 4.4.1](#441)
    
    _Inference algorithms_: custom-proposal importance resampling, resimulation MH
    

<p style="color:blue"> Please choose at least ONE of the following sections to read and do problems in: </p>

5. **Inference algorithms as generative functions**
    
    Can we rewrite the inference algorithms from previous sections to be generative functions themselves? (Spoiler alert: yes.) When we do, we can analyze the probable behavior _of our inference algorithms_: for instance, to compute estimates of their accuracy. 

    _Algorithms_: AIDE
    

6. **Modeling DSLs with custom inference engines**
    
    
    
7. **Syntactic transformations**

   Metaprob comes with a utility that desugars generative functions down to a small core language, involving only literals, variables, `if`, `quote`, and `gen`. Within this smaller core language, we can use PL techniques to accomplish interesting inference tasks statically.
   

8. **Miniprob: Implementing a featherweight PPL**

    In earlier sections, we have focused on implementing a standard inference library on top of a small core language, Metaprob. Here, we'll look at how that small core is implemented: in a series of guided exercises, you will write a 30-line implementation of "Miniprob," a slightly simplified version of Metaprob.

**TODO: Add problem** animating importance sampling for curve-fitting as n particles increases, vs. parallel Markov Chains.

<a name="1"></a>
# 1) Why do we need a language?

---

**Goals of this section**
- We have seen that simulators can represent probabilistic models.
- Our goal in this section will be to see how far we can take this idea without a _probabilistic programming language_, using ordinary Clojure.
- We introduce three inference tasks and models for them: Bayesian model selection, curve fitting, and spelling correction.
- We implement two general-purpose inference algorithms for Clojure-based simulators: Monte Carlo expectations via forward sampling, and posterior inference via rejection sampling.
- We discuss problems with this approach:
  - When the query changes, the model also has to change.
  - The algorithms we can implement are limited, because they have no access to probabilistic information (such as probability densities) about the distributions the simulators represent.
- This motivates the need for a language that treats our simulator code in a special way.

---

There are many interesting questions that can be phrased using the language of probability:
- How much can I expect my lifetime earnings to improve if I get a college degree?
- What is the probability that this treatment will work for this patient?
- With what confidence can we say that these two documents were produced by the same author?

But before we can begin to answer these questions, we need a _model_: a (sometimes simplified) mathematical description of how we imagine the real-world processes behind these questions (the job market, human biology, authorship) actually work. 

One of the key insights in probabilistic programming is that writing a program to _simulate a process_ is a good way to specify our probabilistic models. You have already seen many examples of this technique in the previous problem set, and in lecture. But it's worth pausing to consider: **why do we need a _probabilistic programming language_ to apply this insight? What if we write our simulators in an ordinary language, like Clojure? What kinds of inference can we do?**

<a name="11"></a>
## 1.1) Three example models

We now introduce three modeling and inference problems, and write our models as simulators in Clojure.

### Example 1: Model selection / hypothesis testing for coin flips

**Problem**: A friend has a coin. We are unsure which of the following statements holds of the coin:
- The coin is fair (50/50 chance of heads or tails).
- The coin is painted to have heads (or tails) on both sides (100% heads or 100% tails), with each of these possibilities equally likely.
- The coin is biased with some unknown weight (there is some probability, which could be anywhere between 0 and 1, of coming up heads), and the unknown weight is itself uniformly distributed in the unit interval.

We observe the coin being flipped some number of times. Our goal is to infer which of the above three statements is true.


**Modeling approach**: Our simulator will work as follows:
- Randomly sample one of the three possibilities above.
- Choose $p$, the probability of heads, according to which of the options was selected.
- Flip a $p$-weighted coin $n$ times.

In [ ]:
;; Flip a fair coin n times: p = 0.5
(defn coin-model-1 [n] 
  (let [p 0.5]
    (replicate n (fn [] (flip p)))))

;; Flip a painted coin n times: p = 0 or p = 1
(defn coin-model-2 [n] 
  (let [p (uniform-discrete [0 1])]
    (replicate n (fn [] (flip p)))))

;; Flip a biased coin n times: p ~ Uniform(0, 1)
(defn coin-model-3 [n]
  (let [p (uniform 0 1)]
    (replicate n (fn [] (flip p)))))

;; Our simulator first selects an underlying model, then flips the coin n times.
(defn coin-model [n] 
  (let [which-model (uniform-discrete [1 2 3])
        flip-coins (nth [coin-model-1 coin-model-2 coin-model-3] (- which-model 1))]
     (flip-coins n)))

If you are new to Clojure, a few things are worth noting:

1. Function calls, which in Python and Julia are written `f(x, y)`, are written in Clojure as `(f x y)`. One consequence of this is that _parentheses cannot be used liberally_: they _always_ denote function application. That is, while `4` represents the number 4, the expression `(4)` does not, and is invalid Clojure: Clojure will think you are trying to call the function `4` on no arguments. Similarly, `(f (x) (y))` is very different from `(f x y)`: in the first version, `x` and `y` are themselves functions, each called with zero arguments.

2. We do not use variable assignment in Clojure. Instead of the Pythonic
```python
  p = uniform(0, 1)
  flips = replicate(n, lambda: flip(p))
  return flips
```
we use `(let [name1 value1 ... name-n value-n] body...)` to create local variables:
```clojure
   (let [p (uniform 0 1)
         flips (replicate n (fn [] (flip p)))]
    flips)
```

3. Vectors are created with square brackets, and are indexed into using `nth` (with zero-based indexing, unlike Julia). So `(nth [5 6 9] 2)` is 9.

In [ ]:
;; Run the simulator with 5 coin flips:
(coin-model 5)

### Example 2: Curve-fitting

**Problem**: We are given some data points $(x, y)$ in the plane. We believe there is a polynomial relationship between $x$ and $y$, and we would like to find the polynomial that explains this relationship, keeping in mind that (a) some of the observed points may be outliers, and (b) there may be measurement error even for the inliers.

**Modeling approach**: We will write a model (a data-simulating function in Clojure) that works as follows:
   - Randomly sample a polynomial $f$ (i.e., with random degree and coefficients).
   - Randomly sample a measurement noise level $\sigma$: how far do we expect the observed $y$ values of inliers to be from $f(x)$?
   - Randomly sample an outlier frequency $p$ between $0$ and $1$: what percentage of our observations do we expect to be outliers?
   - For each input point $x_i$, randomly choose whether or not it is an outlier ($o_i$), using a Bernoulli($p$) distribution.
   - If an outlier, sample $y_i$ from a wide base distribution ($\mathcal{N}(0, 10)$). Otherwise, sample $y_i$ from $\mathcal{N}(f(x_i), \sigma)$.

In [ ]:
;; Sample a random polynomial `f` with degree 0, 1, 2, or 3
(defn random-polynomial []
  (let [degree (uniform-discrete [0 1 2 3])
        coeffs (replicate (+ 1 degree) (fn [] (gaussian 0 1)))]
      (fn [x]
        (reduce + 0 (map-indexed (fn [i c] (* c (expt x i))) coeffs)))))

;; Change a deterministic function into a noisy one that also sometimes
;; samples outliers.
(defn noisy-version [f]
    (let [noise-level (gamma 1 1)
          prob-outlier (beta 1 10)]
        (fn [x]
            (if (flip prob-outlier)
                (gaussian 0 10)
                (gaussian (f x) noise-level)))))

;; Given a list of xs, create a list of ys that are related
;; via a noisy polynomial relationship
(defn curve-model [xs]
    (map (noisy-version (random-polynomial)) xs))

We make a few more remarks for those new to Clojure:

1. `random-polynomial` and `noisy-version` are both _higher-order_ functions. The function `random-polynomial` itself returns a _function_ representing the sampled polynomial: that returned function takes in an $x$ value and itself returns $f(x)$. The function `noisy-version` both _accepts_ a function as its argument $f$, and _returns_ a function. The returned function is a noisy version of $f$: on an input $x$, it either generates a completely random point (an outlier), or a noisy measurement of $f(x)$.


2. If you haven't seen `reduce` before, it's worth taking a look at its [documentation](https://clojuredocs.org/clojure.core/reduce). Other functions used here include [map](https://clojuredocs.org/clojure.core/map) and [map-indexed](https://clojuredocs.org/clojure.core/map-indexed). When you feel yourself reaching for a for or while loop, these functions may be what you want.

3. The body of `curve-model`, the final function, could also have been written:

```clojure
(let [underlying-curve (random-polynomial)
      noisy-curve (noisy-version underlying-curve)
      ys (map noisy-curve xs)]
  ys)
```

In [ ]:
;; Try running curve-model on some x values
(curve-model [-5 -4 -3 -2 -1 0 1 2 3 4 5])

### Example 3: Spelling correction

**Problem**: We are examining results from an online survey of Washington State residents and notice that participants have made occasional typos, for example when typing in a home address. Given, say, a potentially typo-ridden city name, our goal is to infer the original string (the correct city name).

**Modeling approach**: Our model (data simulator) will work as follows:
- Randomly sample a city, with probability proportional to its population
- Randomly add typos to the city:
    1. Flip a coin to decide whether to add a typo. With probability 0.9, do _not_ add one, and return the string as is.
    2. If we decided to add a typo, choose what kind (deletion, insertion, or substitution)
    3. Choose an index at which to introduce the typo, and if necessary (i.e., if we are doing an insertion or substitution), a letter to add.
    4. Apply the typo.
    5. Go back to step 1, to decide whether to add _another_ typo.

In [ ]:
;; A list of all English letters
(def alphabet '("a" "b" "c" "d" "e" "f" "g" "h" "i" "j" "k" "l" "m" "n" "o" "p" "q" "r" "s" "t" "u" "v" "w" "x" "y" "z"))

;; Helper functions for string manipulation
(defn delete-at-index [s idx] (str (subs s 0 idx) (subs s (+ 1 idx))))
(defn insert-at-index [s idx c] (str (subs s 0 idx) c (subs s idx)))
(defn replace-at-index [s idx c] (str (subs s 0 idx) c (subs s (+ 1 idx))))

;; Add a single random typo to a string
(defn add-error [true-string]
  (let [error-type (uniform-discrete [:deletion :insertion :substitution])
        highest-possible-index (+ (count true-string) (if (= error-type :insertion) 1 0))
        error-index (uniform-discrete (range highest-possible-index))]
    (cond 
      (= error-type :deletion) (delete-at-index true-string error-index)
      (= error-type :insertion) (insert-at-index true-string error-index (uniform-discrete alphabet))
      (= error-type :substitution) (replace-at-index true-string error-index (uniform-discrete alphabet)))))

;; Randomly add zero or more typos to a string
(defn add-errors [true-string]
  (if (flip 0.9) 
      true-string 
      (recur (add-error true-string))))

;; Washington State cities:
(def washington-cities {"aberdeen" 16255, "bellingham" 83365, "bremerton" 38572, "everett" 106736, "richland" 53019, "seattle" 668342, "spokane" 212052, "tacoma" 205159, "vancouver" 169294, "yakima" 93357})

;; Full model: pick a city, then maybe add typos:
(defn spelling-model []
    (add-errors (categorical washington-cities)))

The `categorical` function accepts either (a) a vector of (unnormalized) probabilities, or (b) a hashmap whose keys are possibiltiies and whose values are their unnormalized probabilities, and returns (a) a randomly sampled index into the vector, or (b) a randomly sampled key of the map, respectively. Here, we use the second form.

Clojure notes:


1. The `cond` macro has the following syntax:

```clojure
(cond
  condition-1  result-1
  condition-2  result-2
  ... ...
  condition-n  result-n)
```
and checks each condition in order until it finds the first one that matches. It then evaluates the corresponding result expression. You can add a "default" case to the end by making the last condition the constant `true`.

2. The Clojure keyword `recur` is used to make a "tail-recursive" call to the enclosing function: `(recur (add-error true-string))` is equivalent to `(add-errors (add-error true-string))`, and serves to take the result of adding _one_ typo (`(add-error true-string)`), and maybe add _more_ typos (`(add-errors ...)`) to _it_.



In [ ]:
;; Get a sense for the types of things this model produces. Note that typos are relatively rare.
(replicate 20 #(spelling-model))

<a name="12"></a>
## 1.2) Simulation-based inference

We now have models, but have yet to do any inference. Because our models are ordinary Clojure functions, all we can do with them is run them (potentially many, many times). In this section, we'll look at the kinds of questions we can answer with this style of straightforward simulation.

<a name="121"></a>
<h3 style="color:blue">Problem 1.2.1: Forward Sampling for Expectations</h3>

One of the simplest kinds of queries just asks for the expectation of a random variable:
- My friend tells me they will flip their coin 100 times. How many coins should I expect to come up heads?
- Under our spelling model, what is the expected length of a name in the city field?

(These are admittedly not very interesting queries, but we have to start somewhere!)

We can estimate answers to these questions by following a simple forward sampling procedure:
- Run the simulator many times.
- Compute the statistic of interest (number of heads, length of string) for each run.
- Average the results.

**Problem:** Write a procedure `(monte-carlo-expectation p f n)` that estimates the expectation of a function `f(x)` when `x` is distributed according to the model `p` (here, `p` is a simulator with zero arguments; when we run it, we get a new independent sample of `x`). The parameter `n` determines how many samples we should obtain and then use in forming the estimate.

In [ ]:
;; -------------------------------------------- ;;
;; ----------------- PROBLEM ------------------ ;;
;; -------------------------------------------- ;;

;; Replace the '... with your own code.
(defn monte-carlo-expectation [p f n]
  '...)

;; Here, we use the function to answer the two queries posed in the problem description:
(defn count-heads [l] (count (filter true? l)))

(println (str "Expected number of heads:" (monte-carlo-expectation (fn [] (coin-model 100)) count-heads 1000.)))
(println (str "Expected length of observed city name: " (monte-carlo-expectation spelling-model count 1000.)))

In [ ]:
;; -------------------------------------------- ;;
;; ----------------- SOLUTION ----------------- ;;
;; -------------------------------------------- ;;
(defn monte-carlo-expectation [p f n]
  (/ (reduce + (map f (replicate n p))) n))

;; Here, we use the function to answer the two queries posed in the problem description:
(defn count-heads [l] (count (filter true? l)))

(println (str "Expected number of heads: " (monte-carlo-expectation (fn [] (coin-model 100)) count-heads 1000.)))
(println (str "Expected length of observed city name: " (monte-carlo-expectation spelling-model count 1000.)))

<a name="122"></a>
<h3 style="color:blue">Problem 1.2.2: Estimating Probabilities</h3>

Using the `monte-carlo-expectation` function you wrote above, we can write a function `estimate-probability` that is able to estimate the probability of some event (e.g., seeing more than 60 heads). The key idea is to notice that $\mathbb{P}(A) = \mathbb{E}[\mathbb{1}_A(x)]$, where $\mathbb{1}_A$ is the _indicator function_ that returns 1 if its argument is in $A$ and 0 otherwise.

1. Define the higher-order function `indicator`, which given a predicate $p$ (i.e., a function of one argument that returns a Boolean), returns $\mathbb{1}_p$, the function that, on input $x$, returns $1$ if $p(x)$ was true, and $0$ otherwise.

2. We have provided a definition of `estimate-probability` that depends on `indicator`. It accepts a `model`, a `predicate` on the output of the model, and a number `n` of samples to do in forming an estimate. It returns an estimate of $\mathbb{P}_{x \sim \text{model}}(p(x) = \text{True})$. Take a moment to understand why `estimate-probability` works.

3. Use `estimate-probability` to:
    - Estimate the probability under `model = (fn [] (coin-model-1 100))` (flipping a fair coin 100 times) that we see more than 60 heads.
    - Do the same for `coin-model-2`, `coin-model-3`, and the full `coin-model`.
    - Estimate the probability that the observed city name in `spelling-model` has at least one typo -- that is, that the value returned by `spelling-model` is not in the original `washington-cities` hashmap. (Hint: Clojure's `(contains? m k)` checks whether a hashmap `m` contains a certain key `k`.)

In [ ]:
;; -------------------------------------------- ;;
;; ----------------- PROBLEM ------------------ ;;
;; -------------------------------------------- ;;

;; Part 1: Define the `indicator` function
(defn indicator [predicate] '...)

;; Part 2: Read and understand the estimate-probability function
(defn estimate-probability [model predicate n]
  (monte-carlo-expectation model (indicator predicate) n))

;; Part 3: Use the estimate-probability function to compute several estimates
;; *Remove the tick-mark before (estimate-probability ...) and replace the ...*

(println (str "With fair coin (model 1): "  
              '(estimate-probability (fn [] (coin-model-1 100)) 
                                     (fn [results] ...) 
                                     10000.)))
(println (str "With painted coin (model 2): "  
              '(estimate-probability ...
                                     (fn [results] ...) 
                                     10000.)))
(println (str "With biased coin (model 3): "  
              '(estimate-probability ... 
                                     (fn [results] ...) 
                                     10000.)))

(println (str "With unknown coin (coin-model): "  
              '(estimate-probability ...
                                     (fn [results] ...)
                                     10000.)))
                                                            
;; Probability of error in city model:
(println (str "Probability of some typo: " 
              '(estimate-probability
                   ...
                   (fn [result] ...) 
                   10000.)))

In [ ]:
;; -------------------------------------------- ;;
;; ----------------- SOLUTION ----------------- ;;
;; -------------------------------------------- ;;
(defn indicator [predicate] #(if (predicate %) 1 0))
(defn estimate-probability [model predicate n]
  (monte-carlo-expectation model (indicator predicate) n))

;; Probability of getting more than 60 heads
(println (str "With fair coin: "  (estimate-probability #(coin-model-1 100) #(> (count-heads %) 60) 10000.)))
(println (str "With painted coin: "  (estimate-probability #(coin-model-2 100) #(> (count-heads %) 60) 10000.)))
(println (str "With biased coin: "  (estimate-probability #(coin-model-3 100) #(> (count-heads %) 60) 10000.)))
(println (str "With full model (should be average of above three): " (estimate-probability #(coin-model 100) #(> (count-heads %) 60) 10000.)))

;; Probability of error in city model:
(println (str "Probability of some typo: " (estimate-probability spelling-model #(not (contains? washington-cities %)) 10000.)))

<a name="123"></a>
<h3 style="color:blue">Problem 1.2.3: Rejection Sampling</h3>

Most interesting inference queries involve _conditioning_. For instance: _Given_ that $y(0)$ is near $0$, $y(1)$ is near $1$, $y(2)$ is near $4$ and $y(3)$ is near $9$, what are likely values for $y(4)$ and $y(5)$?

How can we answer these questions if all we have is a simulator? Here is one method, called rejection sampling:

1. Run the simulator.
2. Check to see if the condition holds of the sample we produced. If so, return it. Otherwise, go back to step 1 and try again.

Write the procedure `rejection-sample`, which accepts a model (a simulator) and a predicate, and keeps sampling $x$ values from the model until one of them satisfies the predicate. Then, use your implementation of `rejection-sample` on the model `(fn [] (curve-model [0 1 2 3 4 5]))` to obtain a vector of `y` values whose first four elements are "close" to $0, 1, 4,$ and $9$ (you can decide exactly how close). Are the last two values close to 16 and 25?

Hint: In Clojure, `(< a x b)` checks that `x` is between `a` and `b`.

In [ ]:
;; -------------------------------------------- ;;
;; ----------------- PROBLEM ------------------ ;;
;; -------------------------------------------- ;;

;; Part 1: Define the `rejection-sample` function
(defn rejection-sample [model predicate]
  '...)

;; Part 2: prediction on quadratic data. 
;; Remove tick-mark before (rejection-sample ...),
;; and replace ... with your own code that implements the desired predicate.
;; Might take 20-30 seconds to run.
(println "Predicting y4 and y5 based on y0,1,2,3:")
(time '(rejection-sample 
           ;; Model:
           (fn [] (curve-model [0 1 2 3 4 5]))
           ;; Predicate:
           (fn [[y0 y1 y2 y3 _ _]]
             ...)))

;; Part 3:
;; Uncomment when you have implemented `rejection-sample`
;; This keeps running `spelling-model` until we observe a
;; particular string (e.g., "seattl"). Answer the free response questions
;; in the cell below, based on the time that each of the following lines
;; takes to run.
(comment
    (println "Times until a certain typoed city is produced:")
    (println "seattl")
    (time (rejection-sample spelling-model #(= % "seattl")))
    (println "settl")
    (time (rejection-sample spelling-model #(= % "settl")))
    (println "seottle")
    (time (rejection-sample spelling-model #(= % "seottle"))))

**Free response:** 
1. Why does it take less time to see "seattl" than "seatl"?
2. Why does it take less time to see "seattl" than "seottle"?

In [ ]:
;; -------------------------------------------- ;;
;; ----------------- SOLUTION ----------------- ;;
;; -------------------------------------------- ;;

;; Part 1: Define the `rejection-sample` function
(defn rejection-sample [model predicate]
  (let [result (model)]
      (if (predicate result)
          result
          (rejection-sample model predicate))))

;; Prediction on quadratic data
(println "Predicting next points in a quadratic dataset")
(time (println (rejection-sample (fn [] (curve-model [0 1 2 3 4 5]))
                 (fn [[y0 y1 y2 y3 _ _]]
                     (and (< -0.1 y0 0.1) (< 0.9 y1 1.1) (< 3.9 y2 4.1) (< 8.9 y3 9.1))))))
(println "Times until a certain typoed city is produced:")
(println "seattl")
(time (rejection-sample spelling-model #(= % "seattl")))
(println "seatl")
(time (rejection-sample spelling-model #(= % "seatl")))
(println "seottle")
(time (rejection-sample spelling-model #(= % "seottle")))

**Free response solution**
1. The edit distance to "seattl" from "seattle" is shorter, and strings at shorter edit distances are in general more probable in our model.
2. The probability of "seattl" is $\mathbb{P}(\text{make typo}) \cdot \mathbb{P}(\texttt{:delete}) \cdot \mathbb{P}(\texttt{index} = 6) \cdot \mathbb{P}(\text{don't make another typo})$.
   The probability of "seottle" is $\mathbb{P}(\text{make typo}) \cdot \mathbb{P}(\texttt{:substitute}) \cdot \mathbb{P}(\texttt{index} = 2) \cdot \mathbb{P}(\textbf{add o}) \cdot \mathbb{P}(\text{don't make another typo})$.
   So "seottle" is less likely because it has to make more choices (of which letter to replace 'a' with).

<a name="124"> </a>
<h3 style="color:blue">Problem 1.2.4: Rejection Sampling based on Internal Choices</h3>

Suppose we want to answer these questions:
- In `spelling-model`, what was the likely true city if the observed city was "sattle"?
- In `coin-model`, which of the three coin models was most likely used if, when run, I observed more than 60 heads?
- In `curve-model`, what was the degree of the polynomial if all my `y` values were observed to be near 0?

An annoyance with the approach that we've taken so far is that when we run `curve-model`, for example, we only get our final `ys` out, not the degree of the sampled polynomial. Even though we could use rejection sampling to keep running our simulator until all the `y` values are close to 0, we would have no way of inspecting the random polynomial degree that was chosen during the successful run. In order to _get_ the degree, we have to modify our code to _return_ it, in addition to the generated vector of `ys`.

In each of the three cells below, modify the code for each model so that the (provided) rejection sampling queries for the above three questions work.

In [ ]:
;; PROBLEM PART 1: What was the likely true city if the observed city was "sattle"?

;; Change the body of this function so that the rejection sampling
;; query below will answer the question.
(defn modified-spelling-model []
    (add-errors (categorical washington-cities)))

;; REJECTION SAMPLING QUERY
;; When ready to test, delete the tick-mark (').
'(let [[true-city observed-city]
      (rejection-sample 
        modified-spelling-model
        (fn [[true-city observed-city]]
          (= observed-city "sattle")))]
    true-city)

In [ ]:
;; PROBLEM PART 2: Which coin model was likely if I observed > 60 heads (out of 100 flips)?

;; Change the body of this function so that the rejection sampling
;; query below will answer the question.
(defn modified-coin-model [n] 
  (let [which-model 
        (uniform-discrete [1 2 3])
        
        flip-coins
        (nth [coin-model-1 coin-model-2 coin-model-3] (- which-model 1))]
    (flip-coins n)))

;; REJECTION SAMPLING QUERY
;; When ready to test, delete the tick-mark (').
'(let [more-than-sixty-heads?
      (fn [flips] (> (count (filter true? flips)) 60))
      
      ;; The new model returns a vector of degree and ys
      [which-model flips]
      (rejection-sample 
        (fn [] (modified-coin-model 100))
        (fn [[which-model flips]]
          (more-than-sixty-heads? flips)))]
    which-model)

In [ ]:
;; PROBLEM PART 3: What was the degree of the polynomial if the observed `y` values were all near 0?

;; Change the bodies of these functions so that the rejection sampling
;; query at the bottom of this cell will answer the question.

;; Sample a random polynomial `f` with degree 0, 1, 2, or 3
(defn modified-random-polynomial []
  (let [degree (uniform-discrete [0 1 2 3])
        coeffs (replicate (+ 1 degree) (fn [] (gaussian 0 1)))]
       (fn [x]
        (reduce + (map-indexed (fn [i c] (* c (expt x i))) coeffs)))))

;; Given a list of xs, create a list of ys that are related
;; via a noisy polynomial relationship
(defn modified-curve-model [xs]
    (map (noisy-version (modified-random-polynomial)) xs))

;; REJECTION SAMPLING QUERY
(println "Estimated degree of polynomial: ")
;; When ready to test, delete the tick-mark (').
'(let [close-to-zero? 
      (fn [y] (< -0.1 y 0.1))
      
      ;; The new model returns a vector of degree and ys
      [degree ys]
      (rejection-sample 
        (fn [] (modified-curve-model [0 1 2 3]))
        (fn [[degree [y0 y1 y2 y3]]]
          (and (close-to-zero? y0)
               (close-to-zero? y1)
               (close-to-zero? y2)
               (close-to-zero? y3))))]
    degree)

#### Solutions

In [ ]:
;; SOLUTION TO PART 1: What was the likely true city if the observed city was "sattle"?

(defn modified-spelling-model []
    (let [true-city (categorical washington-cities)]
      [true-city (add-errors true-city)]))

(let [[true-city observed-city]
      (rejection-sample 
        modified-spelling-model
        (fn [[true-city observed-city]]
          (= observed-city "sattle")))]
    true-city)

In [ ]:
;; SOLUTION TO PART 2

(defn modified-coin-model [n] 
    (let [which-model (uniform-discrete [1 2 3])
          flip-coins (nth [coin-model-1 coin-model-2 coin-model-3] (- which-model 1))]
        [which-model (flip-coins n)]))

;; REJECTION SAMPLING QUERY
(let [more-than-sixty-heads?
      (fn [flips] (> (count (filter true? flips)) 60))
      
      ;; The new model returns a vector of degree and ys
      [which-model flips]
      (rejection-sample 
        (fn [] (modified-coin-model 100))
        (fn [[which-model flips]]
          (more-than-sixty-heads? flips)))]
    which-model)

In [ ]:
;; SOLUTION TO PART 3

;; Sample a random polynomial `f` with degree 0, 1, 2, or 3
(defn modified-random-polynomial []
  (let [degree (uniform-discrete [0 1 2 3])
        coeffs (replicate (+ 1 degree) (fn [] (gaussian 0 1)))]
       [degree 
        (fn [x]
          (reduce + (map-indexed (fn [i c] (* c (expt x i))) coeffs)))]))

;; Given a list of xs, create a list of ys that are related
;; via a noisy polynomial relationship
(defn modified-curve-model [xs]
    (let [[degree f] (modified-random-polynomial)]
      [degree (map (noisy-version f) xs)]))



;; GOAL: Run the following rejection sampling query
(println "Estimated degree of polynomial: ")
(let [close-to-zero? 
      (fn [y] (< -0.01 y 0.01))
      
      ;; The new model returns a vector of degree and ys
      [degree ys]
      (rejection-sample 
        (fn [] (modified-curve-model [0 1 2 3]))
        (fn [[degree [y0 y1 y2 y3]]]
          (and (close-to-zero? y0)
               (close-to-zero? y1)
               (close-to-zero? y2)
               (close-to-zero? y3))))]
    degree)

<a name="13"></a>
## 1.3) Summary

With black-box simulators as models, we can pose and solve some modeling and inference problems. But there are several reasons to be dissatisfied with this approach:

1. **Slow results:** Because rejection sampling involves running the simulator repeatedly until some condition is satisfied, its running time can be hard to predict. When the condition is unlikely under the model, rejection sampling can take a very long time to produce a result.
2. **Changing the query requires changes to the model:** As we saw in the last exercise, expressing a new query may require editing the model code. We'd like to write our model once, and use it for an expansive set of queries.

In the next section, we'll see how a _probabilistic programming language_ can help resolve both these issues.

<a name="2"></a>
# 2) Metaprob generative functions: Intuition

---

**Goals of this section**
- A probabilistic programming langauge compiles our simulation code into _generative functions_, objects that support a broader range of operations than "run."
- Different langauges introduce different interfaces.
- Metaprob's interface enables us to:
    - Run a generative function forward, just like before. So our old rejection sampling and mc-expectation algorithms still work.
    - Obtain a _trace_ of a generative function. (In this section we describe the trace data structure and operations on it, including `partition-trace`.)
    - Get the probability of a trace.
    - _Estimate_ the probability of a partial trace.
    - _Infer_ the missing values in the trace.
- We describe the math of `infer-and-score`, by introducing the idea of `p(•)/q(•)`.
- To create generative functions, we can either use `make-primitive` or `(gen {:tracing-with t} [...] ...)`. We introduce the syntax for creating generative functions in depth, and connect it to Gen.
- We then translate the examples from section 1 into Metaprob from Clojure -- making them generative functions that trace their choices.
- With access to traces and scores, we are able to implement new inference algorithms:
    - Rejection sampling based on a predicate on traces
    - Likelihood weighting
    - Importance resampling
    - Importance sampling

---

<a name="21"></a>
## 2.1) Tracing and scoring

A core idea in probabilistic programming is that by _instrumenting_ our generative model code, we can do more interesting things with our models during inference. We call such an instrumented model a _generative function_.

An ordinary Clojure function can do _one_ thing: 
- `run`: given arguments, produce a return value

A _Metaprob generative function_, on the other hand, can do _two_ things:
- `run-as-clojure-function`: given arguments, produce a return value
- `infer-and-score`: given arguments and an _observation trace_, produce a return value, a _complete trace_, and a _score_. (This may remind you of `generate` in Gen.)



Let's break this down.

Here is a simple Clojure function that simulates a generative process. We might interpret it as sampling an activity that an MIT student is engaged in at 10am. First, we flip a biased coin to decide whether it is a weekday. Depending on that, we sample from a categorical distribution with appropriate weights.

In [ ]:
;; Clojure function:
(def clojure-function
  (fn [] 
    (if (flip 5/7)
        (categorical {:working 0.8 :playing 0.1 :sleeping 0.1})
        (categorical {:working 0.1 :playing 0.6 :sleeping 0.3}))))

We can run it, but that's the only thing we can do:

In [ ]:
(clojure-function)

Here is a Metaprob _generative function_ that describes the same generative process. We will go into the syntax in more detail later, but note a few parallels with Gen:
- We create the generative function using a special macro, `gen`, which behaves like `fn` but for generative functions.
- Like in Gen, we annotate each random choice with a _name_, or _address_. (Metaprob is also capable of "autotracing", generating these names automatically; see section 7 for details. Most of the time, we will _want_ to choose our own names for the random choices.)
- In Gen, "traced random choices" were made using the syntax `@trace(distribution(arg1, arg2), address)`. In Metaprob, we write `(t address distribution [arg1 arg2])`. (In fact, the name `t` is customizable — the `{:tracing-with t}` clause is what binds the name `t` as this generative function's _tracer_.) Like in Gen, this expression returns the sampled value, but also causes an entry to be added to the generative function's _trace_.

In [ ]:
;; Metaprob generative function:
(def metaprob-generative-function
  (gen {:tracing-with t} []
    (if (t "is-weekday?" flip [5/7])
        (t "morning-activity" categorical [{:working 0.8 :playing 0.1 :sleeping 0.1}])
        (t "morning-activity" categorical [{:working 0.1 :playing 0.6 :sleeping 0.3}]))))

Like the Clojure function, the Metaprob generative function can also be run:

In [ ]:
(metaprob-generative-function)

Unlike the Clojure function, the Metaprob generative function _also_ supports an `infer-and-score` operation. Here is the simplest way of using it:

In [ ]:
;; pprint is used for "pretty printing" -- what matters here is the call to infer-and-score.
(pprint (infer-and-score :procedure metaprob-generative-function))

One way to think about `infer-and-score` is that it is doing an "instrumented run" of the generative function. Instead of just returning a sample `v`, we now get back a vector `[v t s]` with _three parts_:
- **The sampled value `v`:** The first element of the returned vector is the value produced by this run of `metaprob-generative-function`. (Of course, if we cared _only_ about this first element, we would not have used `infer-and-score`; we can get `v` just by running `metaprob-generative-function` directly.)
- **The execution trace `t`:** The second element of the returned vector is the _execution trace_ of this run of `metaprob-generative-function`. It contains an entry for each random choice that was made. (The set of possible traces of our generative function can be thought of as the _sample space_ of the distribution it represents; a single trace, then, is a sampled point in that sample space.) We can use `plot-trace` to visualize execution traces:

In [ ]:
(def sample-trace
  (let [[v t s] (infer-and-score :procedure metaprob-generative-function)]
     t))

;; Plot t, the trace, in a 200x200 pixel figure
(plot-trace sample-trace 200)

We use a tree-like diagram to represent traces because they can be hierarchical, as we will see a bit later on.

- **The score `s`**: The third element of the returned vector is a _score_. There are multiple ways of understanding the score, one of which is to view it as the log of an estimate of the probability of some set of _observations_. But we haven't discussed "observations" yet, nor passed any to `infer-and-score`, and so the score is 0. We'll come back to this in a bit, after we've taken a closer look at the trace datatype.

### The trace datatype

A trace is a tree-like data structure. Every trace
- has a root node (the leftmost gray box in the diagram above);
- optionally stores a value at the root node (displayed inside that gray box); and
- has zero or more named subtraces (connected to the root by thin edges in the diagrams above). Each subtrace has its own root node and, potentially, its own named subtraces.

Subtraces may be named with strings, numbers, or booleans. The name of each subtrace appears above its root node in the diagram above.

Concretely, traces are Clojure hashmaps. The root node's value, if it has one, is stored at the key `:value`; the other keys store the named subtraces.

Here is a trace with only a single root node and no named subtraces:

In [ ]:
(plot-trace {:value 4} 
            50) ; draw the trace in a 50x50 figure

Traces like this usually come from primitive generative functions:

In [ ]:
(let [[v t s] (infer-and-score :procedure uniform
                               ;; Use the :inputs option to specify a list of
                               ;; arguments to the generative function--in this
                               ;; case, we are calling (uniform 7 15) to generate
                               ;; a random number between 7 and 15
                               :inputs [7 15])]
    (plot-trace t 50))

Here is a trace with no root value, but with three subtraces, each of which has a root value:

In [ ]:
; Trace with subtraces with root values
(plot-trace {"Massachusetts" {:value "Boston"}, 
             "California" {:value "Sacramento"}, 
             "New York" {:value "New York"}} 300)

Traces like this come from simple generative functions that make a couple random choices, like the one we saw above. That trace had two subtraces:

In [ ]:
(plot-trace sample-trace 200)

We can use `trace-has-value?` and `trace-value` to check if a trace has a value at its root node, and if so, to pull out and return the value:

In [ ]:
(trace-has-value? sample-trace)

In [ ]:
(trace-has-value? {:value 4})

In [ ]:
(trace-value {:value 4})

We can use `trace-subtrace` to pull out a subtrace at an address, and `trace-has-subtrace?` to check for the existence of a subtrace:

In [ ]:
(trace-subtrace sample-trace "is-weekday?")

In [ ]:
(trace-subtrace sample-trace "morning-activity")

In [ ]:
;; We do not have an "is-weekend?" subtrace -- only "is-weekday?"
(trace-has-subtrace? sample-trace "is-weekend?")

In [ ]:
(trace-value (trace-subtrace sample-trace "morning-activity"))

As a shortcut, instead of writing `(trace-value (trace-subtrace t address))`, we can write `(trace-value t address)`:

In [ ]:
(trace-value sample-trace "morning-activity")

So far, we have seen strings like `"morning-activity"` as addresses. An **address** can also be a (potentially empty) _list_ of subtrace names.

In [ ]:
;; '() is the empty list, and is the _address_ of the root node.
(trace-value {:value 4} '()) 

In [ ]:
;; '("morning-activity") is a one-element list, and is equivalent (as an address) to "morning-activity"
(trace-value sample-trace '("morning-activity"))

We can get all the addresses at which a trace has values by using the `addresses-of` function:

In [ ]:
(addresses-of sample-trace)

Let's look at a more interesting trace. The following generative function calls our first generative function twice:

In [ ]:
(def two-days-in-the-life
  (gen {:tracing-with t} []
    (str "When you first visited, I was "
         (t "first-day" metaprob-generative-function [])
         ", and when you next visited, I was "
         (t "second-day" metaprob-generative-function []))))

In [ ]:
(two-days-in-the-life)

In [ ]:
(def two-days-trace
  (let [[_ t _] (infer-and-score :procedure two-days-in-the-life)] t))

(pprint two-days-trace)

In [ ]:
(plot-trace two-days-trace 300)

The trace is now hierarchical! Our `two-days-trace` has two subtraces: `first-day` and `second-day`. Because each was generated by a call to `metaprob-generative-function`, they share the same structure, as traces with `"is-weekday?"` and `"morning-activity"` keys.

In [ ]:
(addresses-of two-days-trace)

In [ ]:
(trace-subtrace two-days-trace "first-day")

In [ ]:
(trace-subtrace two-days-trace '("first-day" "is-weekday?"))

In [ ]:
(trace-value two-days-trace '("second-day" "morning-activity"))

In [ ]:
;; Although "first-day" is a subtrace of two-days-trace, there is no value
;; at the root node of that subtrace. Put another way, the gray box labeled with
;; "first-day" has nothing in it, so `trace-has-value?` returns false.
(trace-has-value? two-days-trace "first-day")

A generative function can invoke its tracer (in our examples so far, called `t`) with _any_ address:

In [ ]:
(def two-more-days
  (gen {:tracing-with t} []
    (str "Today, I am "
         ;; Use the empty list, '(), to call `metaprob-generative-function`
         ;; at the root node, i.e., without adding a layer of hierarchy
         (t '() metaprob-generative-function [])
         ;; Use a multi-element list to add multiple layers of hierarchy:
         ". Two layers deeper in the hierarchy, I will be "
         (t '("layer-1" "layer-2") metaprob-generative-function []))))

In [ ]:
(def two-days-trace-2
  (let [[_ t _] (infer-and-score :procedure two-more-days)] t))

(pprint two-days-trace-2)

In [ ]:
(plot-trace two-days-trace-2 400)

Apart from running `infer-and-score`, we can also construct and manipulate traces manually:

In [ ]:
;; Modify a value
(plot-trace (trace-set-value two-days-trace-2 "is-weekday?" true) 400)

Traces are actually immutable, so `trace-set-value` does not "modify" in place, but rather return a new, updated trace. `two-days-trace-2` is still the same trace that it was before:

In [ ]:
(plot-trace two-days-trace-2 400)

If `trace-set-value` is used on an address that doesn't exist yet, the address is created:

In [ ]:
;; Add a new address
(plot-trace (trace-set-value two-days-trace-2 '("is-weekday?" "new-field") true) 400)

In [ ]:
(plot-trace (trace-set-value {} '("a" "long" "address" "for" "a" "value") "my value") 600 100)

There is also `trace-set-subtrace`:

In [ ]:
;; Add a subtrace
(plot-trace (trace-set-subtrace two-days-trace-2 "my-subtrace" {"favorite-number" {:value 7}}) 400)

And `trace-merge`:

In [ ]:
;; Merge two traces into one
(plot-trace (trace-merge
                two-days-trace
                two-days-trace-2))

### What are traces good for?

Now that we have traces, what are they good for?

For one, we can use them to address the issue we saw in exercise [1.2.4](#124): that when we changed our query, we had to change our model.

<a name="211"></a>
<h3 style="color:blue">Problem 2.1.1: Rejection sampling with internal choices, round 2</h3>

Complete the definition of rejection sampling below, which now accepts _three_ arguments:
1. A `model` (just as before)
2. A `predicate` (just as before, but operates on _traces_, not return values)
3. An `address` (new!)

and does the following:

1. Repeatedly run `infer-and-score` until a trace is produced that satisfies `predicate`.
2. Return the value of the sampled trace at the given address.

In [ ]:
;; PROBLEM

(defn rejection-sample-2
    [model predicate address]
  (let [[v sampled-trace s]
        (infer-and-score '...)]
      '...))



;; Test: given that the activity is "working", we should see "is-weekday?" = true
;; in over 85% of runs. Delete tick-mark to run
'(< 85
   (count 
    (filter true? (replicate 100
                     (fn [] (rejection-sample-2 metaprob-generative-function
                                                (fn [tr] (= (trace-value tr "morning-activity") :working))
                                                "is-weekday?"))))))

In [ ]:
;; Solution

(defn rejection-sample-2
    [model predicate address]
  (let [[_ trace _]
        (infer-and-score :procedure model)]
      (if (predicate trace)
          (trace-value trace address)
          (recur model predicate address))))


;; Test: given that the activity is "working", we should see "is-weekday?" = true
;; in over 85% of runs.
(< 85
   (count 
    (filter true? (replicate 100
                     (fn [] (rejection-sample-2 metaprob-generative-function
                                                (fn [tr] (= (trace-value tr "morning-activity") :working))
                                                "is-weekday?"))))))

### Scoring a complete observation trace

Traces can also be used to specify **observations**.

Recall `sample-trace`, which looks like this:

In [ ]:
(plot-trace sample-trace 200)

When calling `infer-and-score`, we can pass in `sample-trace` as an _observation trace_:

In [ ]:
(infer-and-score :procedure metaprob-generative-function
                 :observation-trace sample-trace)

Note that:
- In this execution, the "random" choices were constrained to agree with our observations. `infer-and-score` always ensures this is the case.
- Therefore, the return value was the same as it was when we first obtained `sample-trace`: it has to be.
- The score is now non-zero! When the observation trace $t$ is _valid and complete_ (i.e., is a possible output trace of `infer-and-score` on the function being traced), the score is $\log p(t)$, where $p$ is the probability mass function for `metaprob-generative-function`'s distribution over traces.

Using this, we can answer questions like, "What is the probability that it was a weekend and I was sleeping at 10am?"

In [ ]:
(let [[_ _ score]
      (infer-and-score :procedure metaprob-generative-function
                       :observation-trace {"is-weekday?" {:value false}
                                           "morning-activity" {:value :sleeping}})]
    (exp score))

This is what we would expect:

In [ ]:
(* 2/7 0.3)

### Partial observation traces

We can also pass a _partial_ observation trace to `infer-and-score`. A partial trace is a trace that specifies values at only _some_ addresses:

In [ ]:
(infer-and-score :procedure metaprob-generative-function
                 :observation-trace {"morning-activity" {:value :sleeping}})

`infer-and-score` still constrains execution to match our observation trace. But now:
- It also "fills in" any choices we left blank; here, it has filled in "is-weekday?". By default, it does this by just running the function normally: when it encounters a random choice, `infer-and-score` checks to see if the value for that choice is in the observation trace, and if not, samples it fresh. Later, we will see ways of customizing this behavior. In an ideal world, choices like "is-weekday?" would be sampled exactly from the _posterior_ distribution $p_{\texttt{weekday?}}(\texttt{weekday?} \mid \texttt{activity = :sleeping})$.
- Our old interpretation of the score is no longer quite right. Let $u$ be our partial observation trace. Then we can write $p(u)$ for the _marginal probability_ of $u$: the sum, over all possible _completions_ $t$ of $u$, of $p(t+u)$ (here, the $+$ means "merge"). For example, if $u$ specifies that our morning activity was `:sleeping`, then $p(u)$ would be the total probability of sleeping: $p(\texttt{weekday}, \texttt{:sleeping}) + p(\texttt{weekend}, \texttt{:sleeping}) = \frac{5}{7} \cdot 0.1 + \frac{2}{7} \cdot 0.3$. The score returned by `infer-and-score` when provided with the partial observation trace $u$ is _not_ this quantity, but instead an _unbiased estimate_ of it: if we run `infer-and-score` many times, exponentiate each score, and average them, we should get the actual marginal probability or $u$.
- Here is another way of understanding the score. Let $q(t; u)$ be the distribution on _completions_ $t$ of $u$ induced by calling `infer-and-score` with observation trace $u$. Then when we run `infer-and-score`, it implicitly samples a completion $t$ from $q(t; u)$, then returns the _complete trace_ $t + u$ (where $+$ is "merge") and the score $\log\frac{p(t + u)}{q(t; u)}$. When $u$ _is_ a complete trace, then there is nothing to complete, and $q(t; u) = 1$ is the deterministic distribution that always samples an empty completion $t$. This is why the _score_ returned in the full-observation-trace case can be interpreted as $p(u) = p(u+t) = \frac{p(u+t)}{q(t; u)}$. But in general, we have only that $\mathbb{E}_{t \sim q(\cdot; u)}[\frac{p(t+u)}{q(t; u)}] = \sum_{t} (q(t; u) \frac{p(t+u)}{q(t; u)}) = \sum_{t} p(t+u) = p(u)$.

**TODO: improve bullet points above.**

<a name="212"></a>
<h3 style="color:blue">Problem 2.1.2: Constructing observation traces</h3>



We will soon write a generative function for the coin model we saw in Section 1.1. Its traces will look like this:

In [ ]:
(plot-trace
  {"which-model" {:value 2}
   "p" {:value 0.3}
   0 {:value true}
   1 {:value false}
   2 {:value false}
   3 {:value false}} 300)

Write a `make-coin-observation-trace` function that takes in a list `flips` of booleans, and creates a partial observation trace of the proper shape, constraining the values of the `(count flips)` coin flips to their observed values (true or false). Note that the addresses at which the flips occur are integers, not strings.

In [ ]:
;; Problem

;; Hint: you may find Clojure functions like `reduce` and `map-indexed` useful.
;; Also, remember that you have access to functions like `trace-set-value`.

(defn make-coin-observation-trace
    [flips]
  '...)

In [ ]:
; Solution
(defn make-coin-observation-trace
    [flips]
  (reduce
    (fn [tr [i b]]
      (trace-set-value tr i b))
    {}
    (map-indexed (fn [i x] [i x]) flips)))

<a name="213"></a>
<h3 style="color:blue">Problem 2.1.3: Setting multiple values at once</h3>



Sometimes, we'd like to make multiple changes to a trace at once. We can write

```clojure
(trace-set-value
    (trace-set-value tr addr-1 new-val-1)
    addr-2 new-val-2)
```

but this gets cumbersome. Write `trace-set-values`, which can take in an arbitrary number of addresses and values:

```clojure
(trace-set-values
    tr
    addr-1 new-val-1
    addr-2 new-val-2
    ...)
```

Note that the syntax `(fn [x y & the-others] ...)` creates a function that can accept 2 or more arguments; anything beyond the first two are packaged into a list called `the-others`:

In [ ]:
;; Problem

(defn trace-set-values [trace & args]
  (let [;; Partition the arguments (addr1 val1 addr2 val2 .... addrn valn)
        ;; into pairs, to get ((addr1 val1) (addr2 val2) ... (addrn valn))
        bindings 
        (partition 2 args)]
    
    
    (reduce
      (fn [trace [addr value]] '...)
      trace
      bindings)))

;; Test:
(= (trace-set-values {} "a" 1 "b" 2 '("c" "d") 3)
   {"a" {:value 1}, "b" {:value 2}, "c" {"d" {:value 3}}})

In [ ]:
;; Solution

(defn trace-set-values [trace & args]
  (let [bindings (partition 2 args)]
    (reduce
      (fn [tr [addr value]] (trace-set-value tr addr value))
      trace
      bindings)))

;; Test:
(= (trace-set-values {} "a" 1 "b" 2 '("c" "d") 3)
   {"a" {:value 1}, "b" {:value 2}, "c" {"d" {:value 3}}})

<a name="22"></a>
## 2.2) Primitive generative functions

The simplest Metaprob generative functions are the primitive distributions, like `flip` and `gaussian`. Let's explore how they behave.

When run in Clojure, they sample values from some distribution:

In [ ]:
(flip 0.8)

When traced, their traces are of the form `{:value v}`, where `v` is the sampled value.

In [ ]:
(infer-and-score :procedure flip
                 :inputs [0.8])

When scoring an observation trace, a scorer is invoked to give an exact log probability:

In [ ]:
(infer-and-score :procedure flip
                 :inputs [0.8]
                 :observation-trace {:value true})

In [ ]:
(exp -0.2231435513142097)

New primitives can be created using `make-primitive`, which accepts two Clojure functions as arguments:
- a sampler, which produces samples from the distribution
- a scorer, which accepts `[sampled-value sampler-args]` and returns $\log p(\texttt{sampled-value}; \texttt{sampler-args})$.

For example, `flip` is implemented as
```clojure
(def flip
  (make-primitive
     (fn [p] (< (uniform 0 1) p))
     (fn [b [p]] (if b (log p) (log (- 1 p))))))
```

<a name="23"></a>
## 2.3) Compound generative functions

We can create generative functions that trace at multiple addresses using `gen`:

In [ ]:
(infer-and-score
    :procedure 
    (gen {:tracing-with t} []
      [(t "flip-1" flip [0.5])
       (t "flip-2" flip [0.8])]))

The syntax for a `gen` function is:

```clojure
(gen {:tracing-with TRACER-NAME} [arg1 ... argN]
   body...)
```

Inside the body, `TRACER-NAME` is bound to a procedure that can be used to make traced random choices:

```clojure
(TRACER-NAME address generative-function [gf-arg1 ... gf-argN])
```

<a name="231"></a>
<h3 style="color:blue">Problem 2.3.1: Writing Compound Generative Functions</h3>

Write the following compound generative functions:

1. A function `f` that accepts no arguments and:
    - Samples `p` between 0 and 1 from a `(uniform 0 1)` distribution, at address `"pick-p"`.
    - Flips two coins at addresses `"flip-coin-1"` and `"flip-coin-2"` with probability `p` of heads.
    - Samples a return value at address `"return-value"` from a `(gaussian 0 10)` distribution if both flips were heads, or from a `(gaussian 0 1)` distribution otherwise.


2. A function `g` that accepts one argument, `x` and:
    - Calls `f` at address `"generate-number"` to get return value `r`
    - Flips a coin at address `"final-flip"`, with parameter `p = 0.2` if `(abs x) < (abs r)` and `p = 0.8` otherwise.


3. A function `h` that behaves just like `g`, except that instead of storing `f`'s choices under the hierarchical address `"generate-number"`, `f`'s choices are stored alongside `"final-flip"` at the top level of the trace. That is, the trace of `h` should include addresses `"pick-p"`, `"flip-coin-1"`, `"flip-coin-2"`, `"return-value"`, and `"final-flip"`.

In [ ]:
;; Problem
(def abs #(Math/abs %))

(def f
  (gen {:tracing-with t} []
    '...))

(def g
  (gen {:tracing-with t} [x]
    '...))

(def h
  (gen {:tracing-with t} [x]
    '...))

In [ ]:
;; Solution

(def f
  (gen {:tracing-with t} []
    (let [p (t "pick-p" uniform [0 1])]
      (if (and (t "flip-coin-1" flip [p])
               (t "flip-coin-2" flip [p]))
          (t "return-value" gaussian [0 10])
          (t "return-value" gaussian [0 1])))))

(def g
  (gen {:tracing-with t} [x]
    (let [r (t "generate-number" f [])]
      (t "final-flip" flip [(if (< (abs x) (abs r)) 0.2 0.8)]))))

(def h
  (gen {:tracing-with t} [x]
    (let [r (t '() f [])]
      (t "final-flip" flip [(if (< (abs x) (abs r)) 0.2 0.8)]))))

We now turn to writing more complex generative functions, translating our examples from [Section 1.1](#11):

### Example 1: Model selection for coin flips

Let's start by translating our first example from [Section 1.1](#11): the coin-flipping model.

We'll start by writing generative functions to represent each of the three hypothesized generative processes that might be at work. (Recall that we are trying to decide between three hypotheses about a coin, which we observe is flipped $n$ times.) 

Our first hypothesis is that a _fair_ coin is flipped $n$ times:

In [ ]:
(def coin-model-1 
  (gen {:tracing-with t} [n] 
    (map (fn [i] (t i flip [0.5])) (range n))))

Note that we've used `map` over the list `(range n)` here, instead of `replicate`: this way, we gain access to `i`, the index of the current flip. We use `i` as the _address_ at which the flip's result will be traced. Traces look like this:

In [ ]:
(let [[_ t _] (infer-and-score :procedure coin-model-1 :inputs [3])] (plot-trace t 200))

Our second and third hypotheses are similar, except that they also sample a probability of heads, $p$, chosen uniformly from either the set {0,1} or the interval [0,1], respectively:

In [ ]:
(def coin-model-2 
  (gen {:tracing-with t} [n] 
    (let [p (t "p" uniform-discrete [[0 1]])]
      (map (fn [i] (t i flip [p])) (range n)))))

In [ ]:
(let [[_ t _] (infer-and-score :procedure coin-model-2 :inputs [3])] (plot-trace t 200))

In [ ]:
(def coin-model-3 
  (gen {:tracing-with t} [n] 
    (let [p (t "p" uniform [0 1])]
      (map (fn [i] (t i flip [p])) (range n)))))

In [ ]:
(let [[_ t _] (infer-and-score :procedure coin-model-3 :inputs [3])] (plot-trace t 200))

Finally, we put everything together into a single `coin-model` generative process that makes two traced generative function calls. The first is to `uniform-discrete`, which we trace at the address `"which-model"`: it chooses which of the three models above we'll use. The second is to the chosen coin model itself. We use an address of `'()`, the empty list, so that the call to `coin` is traced without additional hierarchy:

In [ ]:
(def coin-model 
  (gen {:tracing-with t} [n]
    (let [which-model 
          (t "which-model" uniform-discrete [[1 2 3]])
        
          coin 
          (nth [coin-model-1 coin-model-2 coin-model-3] (- which-model 1))]
        (t '() coin [n]))))

In [ ]:
(let [[_ t _] (infer-and-score :procedure coin-model :inputs [3])] (plot-trace t 250))

You can think of `(t '() f args)` as saying: "call `f`, and merge its trace directly into my trace." By contrast, `(t "some-address" f args)` means: "call `f`, and add its trace to mine as a named subtrace under the label `"some-address"`." 

<span style="font-size:0.8em">If we make two different calls at the same address, e.g., `(t "some-address" f args)` and `(t "some-address" g args)`, the traces of `f` and `g` will be merged to form the `"some-address"` subtrace. (In this case, it is an error if the traces of `f` and `g` conflict with one another.)</span>

### Example 2: Curve-fitting

We now turn to the curve-fitting example.

We begin by translating our `random-polynomial` function. 

First, we take the random choice of `degree` and trace it at address `"degree"`, writing: `(t "degree" uniform-discrete [[0 1 2 3]])`. Why `[[0 1 2 3]]` instead of `[0 1 2 3]`? The last argument to `t` is the _list of arguments_ that `uniform-discrete` accepts. `uniform-discrete` accepts a single argument, so the list is one element long: `[...]`. But that argument happens to also be a list, `[0 1 2 3]`, so all together, we have `[[0 1 2 3]]`. 

Next, we use the same technique as we saw in `coin-model` to make a series of choices (here, the coefficients), traced at integer addresses. The $i^{th}$ coefficient is traced at address `(list "coeffs" i)`.

We return an ordinary Clojure function representing the polynomial we sampled.

In [ ]:
;; Sample a random polynomial `f` with degree 0, 1, 2, or 3
(def random-polynomial 
  (gen {:tracing-with t} []
    (let [degree (t "degree" uniform-discrete [[0 1 2 3]])
          coeffs (map (fn [i] (t (list "coeffs" i) gaussian [0 1])) (range (+ 1 degree)))]
      (fn [x]
        (reduce + (map-indexed (fn [i c] (* c (expt x i))) coeffs))))))

In [ ]:
;; Visualize a trace of random-polynomial
(let [[_ t _] (infer-and-score :procedure random-polynomial)]
  (plot-trace t 300))

A note about Clojure lists:

- So far, we have seen `'("a" "b" "c")` as one way of creating lists, and `(list "a" "b" "c")` as another. In this case, they are equivalent.
- However, in a "quoted list" (like `'("a" "b" "c")`), _no evaluation_ takes place. So in the above code, writing `'("coeffs" i)` would _not_ work: the variable `i` would not be evaluated.
- If we use a backtick (`` ` ``), instead of a single quote mark (`'`), we can mark certain elements or expressions as "to be evaluated" using `~`: `` `("coeffs" ~i)`` is equivalent to `(list "coeffs" i)`.
- None of these are the same as `["coeffs" i]`, which returns a _vector_, not a _list_ (these are two different types in Clojure, though both support the `nth` operation).

Next, we write our `noisy-version` function, which accepts as input a polynomial (a Clojure function taking a number $x$ to its image $f(x)$), and returns a _noisy version_ of that polynomial: a version that usually returns something _near_ $f(x)$, but sometimes ignores $f(x)$ entirely and generates an outlier.

There are a couple things to note about this generative function:
- At the time that we call `noisy-version`, only two random choices are made: the noise level and the probability of outliers. These are both traced using `noisy-version`'s tracer, `t`.
- The _return value_ of `noisy-version` is a _new_ generative function, which we will later call several times, and which makes its _own_ random choices: `is-outlier?` and `y`. To emphasize that it is a different generative function from `noisy-version` itself, we have named its tracer `u`, and used `u` to trace `"is-outlier?"` and `"y"`.
- We have named the Gaussian random variable `"y"` on both sides of the `if`, so that they are treated as "the same choice" (despite being on two different lines of code).

In [ ]:
;; Change a deterministic function into a noisy one, that also sometimes
;; samples outliers.
(def noisy-version
  (gen {:tracing-with t} [f]
    (let [noise-level (t "noise-level" gamma [1 1])
          prob-outlier (t "prob-outlier" beta [1 10])]
        (gen {:tracing-with u} [x]
            (if (u "is-outlier?" flip [prob-outlier])
                (u "y" gaussian [0 10])
                (u "y" gaussian [(f x) noise-level]))))))

In [ ]:
;; Visualize trace of noisy-version
(let [f (random-polynomial)
      [_ t _] (infer-and-score :procedure noisy-version
                               :inputs [f])]
    (plot-trace t 200))

In [ ]:
; Visualize trace of the function _returned_ by noisy-version:
(let [f (random-polynomial)
      noisy-f (noisy-version f)
      [_ t _] (infer-and-score :procedure noisy-f
                               :inputs [0])]
    (plot-trace t 300))

In [ ]:
;; Visualize a trace of a function that combines the three parts above
(let [combined (gen {:tracing-with t} []
                 (t "sample-datum"
                    ;; Function to call for sample-datum:
                    (t "add-noise" 
                     ;; Function call for add-noise:
                     noisy-version 
                     ;; Inputs to noisy-version:
                     [(t "sample-curve" 
                         ;; Function to call for sample-curve:
                         random-polynomial 
                         ;; Inputs to random-polynomial:
                         [])])
                    
                    ;; Inputs to the above function (for "sample-datum"):
                    [0]))
      [_ t _] (infer-and-score :procedure combined)]
    (plot-trace t 400 400))

Of course, we want to sample at more than one point. Our final generative function, `curve-model`, is like `combined` in the above example, in that it combines the calls to `random-polynomial`, `noisy-version`, and the noisy polynomial itself into one master generative function.

The syntax

```
`("data" ~i)
```

is equivalent to `(list "data" i)`. **TODO: Add reference to quasiquote.**

In [ ]:
;; Given a list of xs, create a list of ys that are related
;; via a noisy polynomial relationship
(def curve-model
  (gen {:tracing-with t} [xs]
    (let [underlying-curve (t "underlying-curve" random-polynomial [])
          noisy-curve (t "add-noise" noisy-version [underlying-curve])]
        (doall (map-indexed (fn [i x] (t `("data" ~i) noisy-curve [x])) xs)))))

In [ ]:
(let [[_ t _] (infer-and-score :procedure curve-model
                               :inputs ['(1 2 3)])]
    (plot-trace t))

In [ ]:
(pprint (infer-and-score :procedure curve-model :inputs ['(1 2 3)]))

In [ ]:
;; Useful helpers for curve-fitting
(defn make-observation-trace 
    [ys]
  {"data" (into {} (map-indexed (fn [i y] [i {"y" {:value y}}]) ys))})

(defn count-points [tr] (count (keys (trace-subtrace tr "data"))))
(defn get-ys [tr] (map (fn [i] (trace-value tr `("data" ~i "y"))) (range (count-points tr))))

### Example 3: Spelling correction

The generative function version of `add-error` is straightforward: it is the same as the original Clojure version, but with every random choice named. Note that this includes the `"new-letter"` choice that shows up in each of the last two lines.

In [ ]:
(def add-error 
  (gen {:tracing-with t} [true-string]
    (let [error-type (t "error-type" uniform-discrete [[:deletion :insertion :substitution]])
          highest-possible-index (+ (count true-string) (if (= error-type :insertion) 1 0))
          error-index (t "error-index" uniform-discrete [(range highest-possible-index)])]
      (cond 
        (= error-type :deletion) (delete-at-index true-string error-index)
        (= error-type :insertion) (insert-at-index true-string error-index (t "new-letter" uniform-discrete [alphabet]))
        (= error-type :substitution) (replace-at-index true-string error-index (t "new-letter" uniform-discrete [alphabet]))))))

In [ ]:
(plot-trace (let [[_ t _] (infer-and-score :procedure add-error :inputs ["seattle"])] t) 200)

The `add-errors` function uses a recursive helper function to keep adding typos until it decides it's done adding errors:

In [ ]:
(def add-errors 
  (gen {:tracing-with t} [true-string]
    (let [helper (fn [current-string error-number]
                     (if (t `(~error-number "done-adding-errors?") flip [0.9])
                         (t "final-string" exactly [current-string])
                         (recur (t error-number add-error [current-string]) (+ error-number 1))))]
       (helper true-string 0))))

In [ ]:
(plot-trace 
    (let [[_ t _] (infer-and-score :procedure add-errors :inputs ["seattle"])] t)
    300)

Finally, we put the pieces together into `spelling-model`:

In [ ]:
;; Washington State cities:
(def washington-cities {"aberdeen" 16255, "bellingham" 83365, "bremerton" 38572, "everett" 106736, "richland" 53019, "seattle" 668342, "spokane" 212052, "tacoma" 205159, "vancouver" 169294, "yakima" 93357})

(def spelling-model
  (gen {:tracing-with t} []
    (let [city (t "true-city" categorical [washington-cities])]
      (t "add-errors" add-errors [city]))))

In [ ]:
(plot-trace (let [[_ t _] (infer-and-score :procedure spelling-model)] t) 500)

<a name="232"></a>
<h3 style="color:blue">Problem 2.3.2: Traces of a model</h3>

Suppose that we run `(infer-and-score :procedure spelling-model)` and get as a return value `"spojane"`. Give two possible traces that could have led to this return value, defining the higher-probability trace as `trace-1` and the lower-probability trace as `trace-2`.

In [ ]:
;; -------------------------------------------- ;;
;; ----------------- PROBLEM ------------------ ;;
;; -------------------------------------------- ;;

;; Trace 1 (higher probability):
(def trace-1
  {'... '...})

;; Trace 2 (lower probability):
(def trace-2
  {'... '...})

;; Testing -- should return true
(let [[v1 t1 s1] (infer-and-score :procedure spelling-model, :observation-trace trace-1)
      [v2 t2 s2] (infer-and-score :procedure spelling-model, :observation-trace trace-2)]
    (and
      ;; Trace 1 returns "spojane"
      (= v1 "spojane")
      ;; Trace 2 returns "spojane"
      (= v2 "spojane")
      ;; Trace 1 was a complete, valid trace
      (= t1 trace-1)
      ;; Trace 2 was a complete, valid trace
      (= t2 trace-2)
      ;; Trace 1 had a higher probability than Trace 2
      (> s1 s2)))

In [ ]:
;; -------------------------------------------- ;;
;; ----------------- SOLUTION ----------------- ;;
;; -------------------------------------------- ;;

;; Trace 1 (higher probability). Substitute "k" for "j" in "spokane"
(def trace-1
  {"true-city" {:value "spokane"}
   "add-errors" {0 {"done-adding-errors?" {:value false},
                    "error-type" {:value :substitution},
                    "error-index" {:value 3},
                    "new-letter" {:value "j"}},
                 1 {"done-adding-errors?" {:value true}}
                 "final-string" {:value "spojane"}}})

;; Trace 2 (lower probability). Delete "k" then add "j" in "spokane"
(def trace-2
  {"true-city" {:value "spokane"}
   "add-errors" {0 {"done-adding-errors?" {:value false},
                    "error-type" {:value :deletion},
                    "error-index" {:value 3}},
                 1 {"done-adding-errors?" {:value false},
                    "error-type" {:value :insertion},
                    "error-index" {:value 3},
                    "new-letter" {:value "j"}},
                 2 {"done-adding-errors?" {:value true}}
                 "final-string" {:value "spojane"}}})

;; Testing:
(let [[v1 t1 s1] (infer-and-score :procedure spelling-model, :observation-trace trace-1)
      [v2 t2 s2] (infer-and-score :procedure spelling-model, :observation-trace trace-2)]
    (and
      (= v1 "spojane")
      (= v2 "spojane")
      (= t1 trace-1)
      (= t2 trace-2)
      (> s1 s2)))

<a name="233"></a>
<h3 style="color:blue">Problem 2.3.3: Bundling a function with its arguments</h3>

In this problem, you will write a utility that we will find useful in the rest of the problem set. Write a function `with-args` that accepts as an argument (a) a generative function `f`, and (b) a vector `args` of arguments to the generative function. Return a new generative function of 0 arguments, which calls `f` on `args` and has the same trace as `f` does. You should only need to write one line of code.

In [ ]:
;; PROBLEM

(defn with-args
  [f args]
  (gen {:tracing-with t} []
    '...))

In [ ]:
;; SOLUTION

(defn with-args
  [f args]
  (gen {:tracing-with t} []
    (t '() f args)))

In [ ]:
;; Example use case (should be true):
(let [[_ t1 _]
      (infer-and-score :procedure coin-model :inputs [5])
      [_ t2 _]
      (infer-and-score :procedure (with-args coin-model [5])
                       :observation-trace t1)]
    (= t1 t2))

<a name="24"></a>
## 2.4) Score-based inference with generative functions

We have seen that tracing makes it easier to pose queries about internal model choices without rewriting our model, and that scoring allows us to formulate exact answers to questions about probabilities of complete traces.

We will now see that scores are useful for other reasons, too: we will implement three closely related inference algorithms that rely heavily on scoring of partial observation traces.

### Algorithm 1: Likelihood weighting

Our first algorithm is _likelihood weighting_, a technique for estimating the probability of an event. For example, we may wish to estitmate the probability that five coin tosses all come up heads under our coin model, or the probability that exactly two typos are introduced under our spelling model.

Recall that when given a partial observation trace $t$, `infer-and-score` returns a score that is the log of an _unbiased estimate_ of $p(t)$, the marginal probability of the choices in $t$. This means that if we run `infer-and-score` many times, exponentiate the scores that result, and average the exponentiated scores, we should get a good estimate of $p(t)$.

Here is an implementation of this algorithm, called "likelihood weighting":

In [ ]:
(defn likelihood-weighting
  [model observations n]
  (let [weights (replicate n 
                    (fn [] 
                      (let [[_ _ s] (infer-and-score :procedure model
                                                     :observation-trace observations)]
                          s)))]
      (exp (- (logsumexp weights) (log n)))))

For example, let's say we want to know the total probability of `:sleeping` as the `"morning-activity"` under our `metaprob-generative-function` model we wrote earlier.

In [ ]:
(likelihood-weighting
    metaprob-generative-function
    {"morning-activity" {:value :sleeping}}
    10000)

Manually calculating the result, we get:

In [ ]:
;; P(Weekend)P(Sleeping|Weekend) + P(Weekday)P(Sleeping|Weekday)
(+ (* 2/7 0.3) (* 5/7 0.1))

Similarly, for `:working`:

In [ ]:
(likelihood-weighting
    metaprob-generative-function
    {"morning-activity" {:value :working}}
    10000)

In [ ]:
(+ (* 2/7 0.1) (* 5/7 0.8))

<a name="241"></a>
<h3 style="color:blue">Problem 2.4.1: Estimating Probabilities with Likelihood Weighting</h3>

Use likelihood weighting to estimate:
- The probability that in `coin-model`, when your the coin is tossed five times, all five come up heads.
- The probability that in `spelling-model`, a user introduces two typos.

In [ ]:
;; PROBLEM

;; Remove tick-marks and fill in ...s

;; Probability of all 5 heads
(println (str "Probability of all five heads: " 
             '(likelihood-weighting 
                   (with-args coin-model [5])
                   ...
                   1000)))

;; Probability of error in city model:
(println (str "Probability of exactly two typos: " 
              '(likelihood-weighting 
                  spelling-model 
                  ... 
                  1000))) ;; Can be reduced without losing accuracy

In [ ]:
;; SOLUTION

;; Probability of all 5 heads
(println (str "Probability of all five heads: " (likelihood-weighting (with-args coin-model [5])
                                                                      {0 {:value true}
                                                                       1 {:value true}
                                                                       2 {:value true}
                                                                       3 {:value true}
                                                                       4 {:value true}}
                                                                      1000)))
;; Probability of error in city model:
(println (str "Probability of exactly two typos: " (likelihood-weighting spelling-model 
                                                                 {"add-errors" {0 {"done-adding-errors?" {:value false}}
                                                                                1 {"done-adding-errors?" {:value false}}
                                                                                2 {"done-adding-errors?" {:value true}}}} 
                                                                 1000.)))

### Algorithm 2: Importance sampling

Suppose that instead of wanting to know the probability of an event, we want, more generally, to know the _conditional expectation_ of some function $f(t)$ given some event $u$. Here, $t$ is a trace sampled from the conditional distribution $p(t|u)$, and $u$ is a partial observation trace representing the event we are conditioning on. For example, we may wish to answer the question: "In `curve-model`, what is the expected coefficient of `x`, given that we saw these `y` values?" The observed $y$ values would be packaged into a partial observation trace $u$, and our $f$ would accept a trace $t$ of `curve-model` and extract the value at `'("coeffs" 1)` to get the coefficient of the $x^1$ term.

The idea in importance sampling is to sample many traces $t_i$ from `infer-and-score`, all with observation trace $u$. These traces will each have a score $\log w_i$ attached. We apply $f$ to each $t_i$, and then take the weighted average of the result: $\frac{\sum_i f(t_i)w_i}{\sum_i w_i}$.


For an argument that this algorithm really does estimate the desired conditional expectation, [this article](https://statweb.stanford.edu/~owen/mc/Ch-var-is.pdf) is a good resource (particularly the section on self-normalized importance sampling). A sketch of the proof goes something like this, reusing the notation from the end of [Section 2.1](#21):

\begin{align*}
\mathbb{E}_{t \sim p(t|u)}[f(t)] =& \sum_t p(t|u)f(t)\\
=& \sum_t q(t; u) \frac{p(t|u)}{q(t; u)} f(t)\\
=& \frac{\sum_t q(t; u) \frac{p(t|u)p(u)}{q(t; u)} f(t)}{p(u)}\\
=& \frac{\mathbb{E}_{t \sim q(t; u)}[f(t)\frac{p(t+u)}{q(t; u)}]}{\mathbb{E}_{t \sim q(t; u)}[\frac{p(t+u)}{q(t;u)}]}\\
\end{align*}

The algorithm is implemented below:

In [ ]:
(defn importance-sample
  [model f observations n]
  (let [particles (replicate n 
                    (fn [] 
                      (let [[_ t s] (infer-and-score :procedure model
                                                     :observation-trace observations)]
                          [(* (exp s) (f t)) s])))
        normalizer (exp (logsumexp (map second particles)))]
      
      (/ (reduce + (map first particles)) normalizer)))

We can now apply it to curve-fitting:

In [ ]:
;; What is the expected degree of the polynomial, given that y(x) = x for x=0,1,2,3,4,5?
(importance-sample
    (with-args curve-model [[0 1 2 3 4 5]])
    (fn [tr] (trace-value tr '("underlying-curve" "degree")))
    {"data" {0 {"y" {:value 0}}, 
             1 {"y" {:value 1}}, 
             2 {"y" {:value 2}}, 
             3 {"y" {:value 3}}, 
             4 {"y" {:value 4}}, 
             5 {"y" {:value 5}}}} 1000)

<a name="242"></a>
<h3 style="color:blue">Problem 2.4.2: Estimating Conditional Probabilities with Importance Sampling</h3>

Importance sampling is a tool for estimating conditional expectations. We can frame the probability of an event $E$ given some observation trace $obs$ as the expectation of the indicator function $\mathbb{1}_E(t)$ when $t \sim \text{model}(\cdot \mid obs)$. Write a function `estimate-probability-is` that uses importance sampling, and the idea of `indicator` functions, to estimate the probability (under some model) of a predicate on traces, given some observations.

In [ ]:
(defn estimate-probability-is
    [model predicate observations n]
  (importance-sample '... '... '... '...))

In [ ]:
;; Solution
(defn estimate-probability-is
    [model predicate observations n]
  (importance-sample model (indicator predicate) observations n))

Now, suppose we are using our `coin-model` and we've seen five heads in a row. We'd like to compare the probabilities of our three different coin models given our observations.

In [ ]:
;; Our observations
(def five-heads-trace {0 {:value true}
                       1 {:value true}
                       2 {:value true}
                       3 {:value true}
                       4 {:value true}})

;; Our model
(def five-coins-model (with-args coin-model [5]))

;; Remove the tick mark and replace the ...s with the proper arguments
(println (str "Probability of fair coin (which-model=1): " '(estimate-probability-is ... ... ... 10000)))
(println (str "Probability of painted coin (which-model=2): " '(estimate-probability-is ... ... ... 10000)))
(println (str "Probability of biased coin (which-model=3): " '(estimate-probability-is ... ... ... 10000)))

In [ ]:
;; Solution

;; Remove the tick mark and replace the ... with the proper arguments
(println (str "Probability of fair coin (which-model=1): " 
              (estimate-probability-is five-coins-model
                                       (fn [t] (= (trace-value t "which-model") 1))
                                       five-heads-trace 10000)))
(println (str "Probability of painted coin (which-model=2): " 
              (estimate-probability-is five-coins-model
                                       (fn [t] (= (trace-value t "which-model") 2))
                                       five-heads-trace 10000)))
(println (str "Probability of biased coin (which-model=3): " 
              (estimate-probability-is five-coins-model
                                       (fn [t] (= (trace-value t "which-model") 3))
                                       five-heads-trace 10000)))

### Algorithm 3: Importance resampling


We now examine one final variation on the idea of "weighted" samples.

Recall that in rejection sampling, the goal was to draw a sample from the _posterior distribution_ on traces, given our observations. Importance sampling, by contrast, simply computes a conditional _expectation_, a point estimate that does not help us characterize the entire _distribution_ of possible answers to our query (and therefore, does not help us characterize our uncertainty).

Importance resampling is a version of importance sampling that yields samples approximately from the posterior. The idea is to generate many weighted traces (called "particles"), then sample one of them at random, where the chance of sampling a particular particle is proportional to its weight. Therefore, particles with higher $\frac{p(t+u)}{q(t; u)}$ scores are more likely to be sampled. Under some conditions, as the number of particles goes to infinity, this method produces  samples from a distribution that converges to $p(t|u)$.

Here is an implementation:

In [ ]:
(defn importance-resample
  [model observations n-particles]
  (let [particles (replicate n-particles
                    (fn [] 
                      (let [[_ t s] (infer-and-score :procedure model
                                                     :observation-trace observations)]
                          [t s])))]
      (first (nth particles (log-categorical (map second particles))))))

<a name="243"></a>
<h3 style="color:blue">Problem 2.4.3: Posterior Inference with Importance Resampling</h3>

Use `importance-resample` to generate traces of `curve-model` conditioned on:
- The observation trace asserting that y(x) = x for x = 0, 1, 2, 3.
- The observation trace asserting that y(x) = x^2 for x = 0, 1, 2, 3.

In [ ]:
;; Problem

;; Observed y=x
(pprint (importance-resample (with-args curve-model [[0 1 2 3 4 5]]) 
                     ;; Replace with your observation trace:
                     {}
                     10000))


;; Observed y=x^2
(pprint (importance-resample (with-args curve-model [[0 1 2 3 4 5]])
                     {}
                     10000))

In [ ]:
;; Solution
(pprint (importance-resample (with-args curve-model [[0 1 2 3 4 5]]) 
                     {"data" {0 {"y" {:value 0}}
                              1 {"y" {:value 1}}
                              2 {"y" {:value 2}}
                              3 {"y" {:value 3}}}}
                     10000))

(pprint (importance-resample (with-args curve-model [[0 1 2 3 4 5]])
                     {"data" {0 {"y" {:value 0}}
                              1 {"y" {:value 1}}
                              2 {"y" {:value 4}}
                              3 {"y" {:value 9}}}}
                     10000))

<a name="3"></a>
# 3) Building a Library for MCMC

---

**Goals of this section**
- So far, we have begun to see how a standard inference library (including rejection sampling, importance resampling, and likelihood weighting) could be implemented.
- But these are all very simple methods: in this section, we'll look at the more complex task of implementing MCMC algorithms using the GFI. We assume familiarity with MH from the Gen problem set.
- The most straightforward to implement is _symmetric proposal MH_. A symmetric proposal is a proposal with the property that moving from t to t' is always just as likely as moving from t' to t. To apply such a proposal, we sample a new trace t' from q(•; t), then compute $\alpha = \min(1, \frac{p(t')}{p(t)})$. With probability $\alpha$ we accept the proposal, and otherwise, we reject.
- For asymmetric proposals, we need to be able to evaluate how likely a given proposal was (and how likely we'd be to go _back_). For these, we need to represent proposals _as generative functions_, so we can analyze them automatically.
- Using symmetric and asymmetric proposals, we can create a good inference algorithm for `curve-model`.
- We can also implement Gibbs sampling for discrete variables who do not affect control flow.
- We still have not seen generic resimulation MH... that will come later!
---

We have begun to see how a standard inference library (including rejection sampling, importance sampling, and likelihood weighting) can be implemented on top of Metaprob's generative function interface. But perhaps this is not too surprising: Metaprob's `infer-and-score` produces exactly the sorts of weights that algorithms like importance sampling use. It is also not especially useful: for many more complex problems, naive importance sampling will be inaccurate and rejection sampling too slow.

In this section, we will extend our growing standard library with functions that help users implement MCMC algorithms for their models. Our library functions will be based on those found in Gen, which you got practice with in Problem Set 1's "Iterative Inference in Gen" notebook. It may be worth revisiting that material for a refresher on the general context in which we'll be working. If you are curious to learn more about the Metropolis Hastings (MH) algorithm, there are also many great sources online (see, e.g., this [primer](http://www.mit.edu/~ilkery/papers/MetropolisHastingsSampling.pdf)). 


Intuitively, we'd like to provide functions that make it easy for users of our library to express algorithms like this (for the curve-fitting problem introduced in the previous two sections):

1. Sample an initial trace from the prior by running `curve-model`.
2. For iter=1 to 10,000:
   1. Try changing the curve's degree, sampling new coefficients if necessary. Then accept or reject this proposal.
   2. Try wiggling each coefficient, by adding a small random number to it. Accept or reject the resulting proposal.
   3. Try guessing new noise parameters (`prob-outlier` and `noise-level`), and accept or reject the proposal.
   4. For each data point: 
       1. Try flipping this point's current outlier classification, and then accept or reject this proposal.
       
A tedious part of writing this algorithm by hand, without the help of a probabilistic programming languages, is computing the acceptance probabilities of each proposal, based on the MH algorithm's "acceptance ratio". If $t$ is the current trace, and $t'$ is a proposed updated trace, the probability with which the proposal should be accepted (rather than rejected) takes the form:

$$\alpha = \min(1, \frac{p(t')}{p(t)} \frac{q(t \leftarrow t')}{q(t' \leftarrow t)})$$

Here, $p$ represents the probability (or density, for continuous distributions) of a complete trace under our model. $q$ is the _proposal distribution_, and $q(t' \leftarrow t)$ is the probability of moving to $t'$ under this proposal when starting at $t$.

Intuitively, then, this ratio says to _accept_ proposals that are _more probable_ under our model (which should make sense), but to be wary about accepting proposals to regions of the trace space that, on future iterations, it will be hard to get back from. 

Our library functions will allow the user to specify how to make the proposals, and automatically determine whether to accept or reject them.

## 3.1) MH with Symmetric Proposals

We begin with a library function for making _symmetric proposals_, which have the property that for any two traces $t$ and $t'$, the probability $q(t' \leftarrow t)$ of proposing $t'$ starting at $t$ is the same as the probability $q(t \leftarrow t')$ of proposing $t$ starting at $t'$. This is a nice property, because when it holds of $q$, the acceptance ratio $\alpha$ simplifies to $\min(1, \frac{p(t')}{p(t)})$. Examples of symmetric proposals include:

1. Update `t` by resampling some random variable in the trace from a uniform distribution.
2. Update `t` by sampling some random noise from a Gaussian distribution centered at 0, and adding it to the current value of some random variable.
3. Update `t` by deterministically flipping some boolean random variable.



<a name="311"></a>
<h3 style="color:blue">Problem 3.1.1: Metropolis-Hastings with Symmetric Proposals</h3>



Our `symmetric-proposer-mh-step` function will accept two arguments:
1. A model, `model`
2. A symmetric "proposer", `symmetric-proposer`, which is a (potentially stochastic) function that accepts a trace as input and returns a proposed trace as output. Users are responsible for ensuring that their proposers are symmetric.

We will return a _new_ trace-to-trace function, which, given a trace $t$, first runs `symmetric-proposer` on it to generate a proposed trace $t'$, then decides whether to return $t$ unchanged or to return the proposed trace $t'$ based on the acceptance ratio $\alpha$.

Here is a partial implementation of the method, which you should complete:

In [ ]:
;; PROBLEM
;; Fill in the '...s below.

;; Inputs: model, a generative function.
;;         symmetric-proposer, a function accepting a current trace and returning a proposed trace
;; Returns: a function that performs a single MH step with the custom proposal, assuming
;;   that the proposer is symmetric.
(defn symmetric-proposer-mh-step [model symmetric-proposer]
  ;; Return a function that performs a single MH step:
  ;; It takes in a current trace, and returns the next iteration's trace
  (fn [current-trace]
    (let [[_ _ current-trace-score] ;; Calculate current trace score (log p(t))
          '...
        
          proposed-trace             ;; Get the new proposed trace (t')
          '...
        
          [_ _ proposed-trace-score] ;; Calculate score of proposed trace (log p(t'))
          '...
        
          log-acceptance-ratio       ;; Calculate log [p(t')/p(t)]
          (min 0 (- proposed-trace-score current-trace-score))]
    
        (if (flip (exp log-acceptance-ratio))  ;; Accept or reject the proposal
           proposed-trace
           current-trace))))

In [ ]:
;; SOLUTION

(defn symmetric-proposer-mh-step [model proposal]
  (fn [current-trace]
    (let [[_ _ current-trace-score]
          (infer-and-score :procedure model
                           :observation-trace current-trace)
        
          proposed-trace 
          (proposal current-trace)
        
          [_ _ proposed-trace-score]
          (infer-and-score :procedure model
                           :observation-trace proposed-trace)
        
          log-acceptance-ratio
          (min 0 (- proposed-trace-score current-trace-score))]
    
        (if (flip (exp log-acceptance-ratio))
           proposed-trace
           current-trace))))

<a name="312"></a>
<h3 style="color:blue">Problem 3.1.2: Gaussian Drift (or "Random Walk") Proposals</h3>



In this exercise, you will add an additional helper function to our standard library, that automatically _creates_ a symmetric proposer that can be passed to `symmetric-proposer-mh-step`.

Write `make-gaussian-drift-proposer`, which takes as input (a) `addresses`, a list of trace addresses updated by the proposer being created, and (b) `width`, a "step size" parameter that is greater than 0, and returns a _function_ that takes in a current trace, and also returns a _proposed_ trace in which random numbers sampled from a `(gaussian 0 width)` distribution have been added to the values at each of the specified `addresses`.

In [ ]:
;; PROBLEM:
(defn make-gaussian-drift-proposer
  [addresses width]
  (fn [current-trace]
    ;; Hint: you may wish to use Clojure's `reduce`
    '...))

In [ ]:
;; SOLUTION
(defn make-gaussian-drift-proposer
  [addresses width]
  (fn [current-trace]
    (reduce
      (fn [tr addr]
        (trace-set-value tr addr (gaussian (trace-value tr addr) width)))
       current-trace
       addresses)))

We can now test both functions you've written, by performing a toy inference task in `curve-model`: 
1. We create a dataset of $x$ and $y$ coordinates for the function $y=x$.
2. We create an observation trace that fixes the $y$s to their values in our dataset, and also fixes the polynomial degree to 1, the noise level to 0.05, and the outlier probability to 0, so that we only have to do inference over the coefficients.
3. Get an initial trace of our model, by calling `infer-and-score`.
4. Create a list of addresses that we are interested in updating with our MH proposper: `'("underlying-curve" "coeffs" 0)` (the intercept) and `'("underlying-curve" "coeffs" 1)` (the slope).
5. Run 100 iterations of `(symmetric-proposer-mh-step (make-gaussian-drift-proposer addresses 0.2))`.

You should see that the inferred coefficients are close to 0 (intercept) and 1 (slope), and that we get this result almost instantly (compared to rejection sampling, which can take a while).

In [ ]:
;; Testing Gaussian Drift and MH.
;; Find correct setting of coefficients for a simple linear
;; dataset in curve-model.
(let [[xs ys] 
      [[0 1 2 3 4 5] [0 1 2 3 4 5]]
      
      ;; Create an observation trace fixing y values.
      ;; Since we are only doing inference on the coefficients,
      ;; we also fix noise to be low (0.05) and degree to be 1.
      observations
      (trace-set-values 
          (make-observation-trace ys)
          '("underlying-curve" "degree") 1
          '("add-noise" "prob-outlier") 0
          '("add-noise" "noise-level") 0.05)
      
      ;; Get an initial trace, subject to observations
      [_ tr _] 
      (infer-and-score :procedure curve-model
                       :inputs [[0 1 2 3 4 5]]
                       :observation-trace observations)
      
      ;; Addresses related to coefficients
      coeff-addresses
      '(("underlying-curve" "coeffs" 0)
        ("underlying-curve" "coeffs" 1))
      
      ;; Make our proposal
      proposer
      (make-gaussian-drift-proposer coeff-addresses 0.1)
      
      ;; Run 500 MH steps
      new-trace 
      (nth (iterate (symmetric-proposer-mh-step (with-args curve-model [xs]) proposer) tr) 100)]
    
    ;; Print results
    (println "ORIGINAL COEFFICIENTS:")
    (pprint (trace-subtrace tr '("underlying-curve" "coeffs")))
    (println "INFERRED COEFFICIENTS:")
    (pprint (trace-subtrace new-trace '("underlying-curve" "coeffs")))
    
    ;; Test:
    (and (< -0.2 (trace-value new-trace '("underlying-curve" "coeffs" 0)) 0.2)
         (<  0.9 (trace-value new-trace '("underlying-curve" "coeffs" 1)) 1.1)))

<a name="313"></a>
<h3 style="color:blue">Problem 3.1.3: A deterministic proposal for outlier classifications</h3>

Now let's add support for inference about outliers. To do so, we will need to create a proposer that can update them. So far, we have only added support for symmetric proposers, so we implement `make-outlier-proposer` to, on input $i$, return a function (a proposer) that takes in a current trace $t$ and returns a version $t'$ where data point $i$'s outlier status has been flipped (from false to true or vice versa).

In [ ]:
;; PROBLEM
(defn make-outlier-proposer [i]
  (fn [old-trace]
    (let [outlier-address (list "data" i "is-outlier?")
          old-outlier-value '...
          new-outlier-value '...]
      
      ;; Construct new trace with flipped outlier status
      '...)))

In [ ]:
;; SOLUTION
(defn make-outlier-proposer [i]
  (fn [old-trace]
    (let [outlier-address (list "data" i "is-outlier?")
          old-outlier-value (trace-value old-trace outlier-address)
          new-outlier-value (not old-outlier-value)]
      
      ;; Construct new trace
      (trace-set-value old-trace outlier-address new-outlier-value))))

We now update our test code from before to also do inference about outliers. Our dataset of $x$s and $y$s also now contains an outlier, $(2, 7)$.

In [ ]:
;; Testing outlier proposal.
;; Find correct setting of coefficients for a simple linear
;; dataset in curve-model, with outliers.
(let [[xs ys] 
      [[0 1 2 3 4 5] [0 1 7 3 4 5]]
      
      ;; Create an observation trace fixing y values.
      ;; Since we are only doing inference on the coefficients,
      ;; we also fix noise to be low (0.05) and degree to be 1.
      observations
      (trace-set-values 
          (make-observation-trace ys)
          '("underlying-curve" "degree") 1
          '("add-noise" "noise-level") 0.05)
      
      ;; Get an initial trace, subject to observations
      [_ tr _] 
      (infer-and-score :procedure curve-model
                       :inputs [[0 1 2 3 4 5]]
                       :observation-trace observations)
      
      ;; Addresses related to coefficients
      coeff-addresses
      '(("underlying-curve" "coeffs" 0)
        ("underlying-curve" "coeffs" 1))
      
      ;; Make our proposal
      coeffs-proposer
      (make-gaussian-drift-proposer coeff-addresses 0.2)
      
      ;; A single step our algorithm is a composition of the coefficients step
      ;; with each outlier step.
      step
      (reduce 
          comp
          (symmetric-proposer-mh-step (with-args curve-model [xs]) coeffs-proposer)
          (map (fn [i] (symmetric-proposer-mh-step (with-args curve-model [xs]) (make-outlier-proposer i)))
               (range (count xs))))
      
      ;; Run 500 MH steps
      new-trace 
      (nth (iterate step tr) 1000)]
    
    ;; Print results
    (println "ORIGINAL COEFFICIENTS:")
    (pprint tr)
    (println "INFERRED COEFFICIENTS:")
    (pprint new-trace)
    
    ;; Test:
    (and (trace-value new-trace '("data" 2 "is-outlier?"))
         (< -0.2 (trace-value new-trace '("underlying-curve" "coeffs" 0)) 0.2)
         (<  0.9 (trace-value new-trace '("underlying-curve" "coeffs" 1)) 1.1)))

## 3.2) MH with asymmetric proposals

Gaussian drift works well for unconstrained continuous parameters like the coefficients in our `curve-model`. But we can't use symmetric proposals for everything. For example, the `noise-level` in `curve-model` must always be kept strictly positive.

We'd like to extend our MH library to be able to handle _asymmetric_ proposals $q$, for which $q(t' \leftarrow t) \neq q(t \leftarrow t')$. The MH acceptance ratio in this case becomes:


$$\min(1, \frac{p(t')}{p(t)} \frac{q(t \leftarrow t')}{q(t' \leftarrow t)})$$

Because the `model` ($p$) is a generative function, we know how to compute $p(t)$ and $p(t')$. But how should we handle the proposal $q$? In the last section, the proposal was an arbitrary Clojure function, which accepted a trace and returned a trace. We didn't need to understand the probability with which it would produce some trace $t$ because we assumed that $q(t \leftarrow t') = q(t' \leftarrow t)$, and so their ratio would be 1. Now, we _do_ need to evaluate that ratio, and it may _not_ be equal to 1.

The solution is to **represent $q$ (the `proposer`) as a generative function, as well.**

In particular, we will require the following of $q$, our proposer:

1. $q$ is a generative function that accepts a trace $t$ and returns a trace $t'$.
2. $q$ itself may make traced random choices.
3. In any run of $q$, if $t'$ differs from $t$ at some address $a$, then the value of $t'$ at $a$ must come from a _random choice that $q$ made at address $a$_.

This third requirement is similar to the requirement placed on custom proposals in Gen, which is that they "act out" the choices they are proposing. That is, in order to propose a new value for some address (say, `'("underlying-curve" "prob-outlier")`), our proposer must _itself_ choose a new value at that address: `(t '("underlying-curve" "prob-outlier") uniform [0 1])`, for example.

Let's get a feel for these requirements by writing proposers that satisfy them.

<a name="321"></a>
<h3 style="color:blue">Problem 3.2.1: An asymmetric proposal for noise and prob-outliers</h3>

We now write a proposer that proposes updates to both the `"noise-level"` parameter and the `"prob-outlier"` parameter at once.

Modify the _generative function definition_ below so that it samples a new noise level (perhaps from a `(gamma 1 1)` distribution) and a new probability of outliers (perhaps from `(beta 1 10)`), then proposes a new trace with those values updated.

In [ ]:
;; PROBLEM
(def noise-proposer
  (gen {:tracing-with t} [old-trace]
    (let [noise-level-addr '("add-noise" "noise-level")
          prob-outlier-addr '("add-noise" "prob-outlier")
          
          ;; Sample new noise level and prob outlier;
          ;; make sure to use (t ...) when sampling,
          ;; so that the _proposer_'s random choices are
          ;; traced at the proper addresses!
          new-noise-level '...
          new-prob-outlier '...]
    
      ;; Using the new values we chose, propose a new 
      ;; trace by modifying old-trace
      '...)))

In [ ]:
;; SOLUTION
(def noise-proposer
  (gen {:tracing-with t} [old-trace]
    (let [noise-level-addr '("add-noise" "noise-level")
          prob-outlier-addr '("add-noise" "prob-outlier")
          new-noise-level (t noise-level-addr gamma [1 1])
          new-prob-outlier (t prob-outlier-addr beta [1 10])]
      (trace-set-values 
          old-trace
          noise-level-addr new-noise-level
          prob-outlier-addr new-prob-outlier))))

<a name="322"></a>
<h3 style="color:blue">Problem 3.2.2: An asymmetric proposal for degree and coefficients</h3>

As another example, write a proposer that, given an old trace, samples a new degree for the old trace, and resamples all coefficients. Make sure that your returned trace has _only_ the relevant values: i.e., if the old trace had degree 3, but your proposed trace has degree 2, make sure that there isn't a value at `'("underlying-curve" "coeffs" 3)` in the proposed trace.

In [ ]:
;; PROBLEM
(def curve-proposer
  (gen {:tracing-with t} [old-trace]
    (let [degree-addr '("underlying-curve" "degree")
          new-degree '...
          coeff-addrs (map (fn [i] `("underlying-curve" "coeffs" ~i)) (range (+ 1 new-degree)))
          new-coeffs (map (fn [addr] '...) coeff-addrs)]
        
      ;; Create new trace by modifying old-trace
      ;; Hint: use `trace-set-subtrace` to replace the entire
      ;; "underlying-curve" subtrace with a new one that contains
      ;; new-degree at address "degree", and each new coefficient
      ;; at address (list "coeffs" i) (for i = 0...new-degree).
      '...)))

In [ ]:
;; SOLUTION
(def curve-proposer
  (gen {:tracing-with t} [old-trace]
    (let [degree-addr '("underlying-curve" "degree")
          new-degree (t degree-addr uniform-discrete [[0 1 2 3]])
          coeff-addrs (map (fn [i] `("underlying-curve" "coeffs" ~i)) (range (+ 1 new-degree)))
          new-coeffs (map (fn [addr] (t addr gaussian [0 1])) coeff-addrs)]
        
      ;; Create new trace by modifying old-trace
      ;; Hint: use `trace-set-subtrace` to replace the entire
      ;; "underlying-curve" subtrace with a new one that contains
      ;; new-degree at address "degree", and each new coefficient
      ;; at address (list "coeffs" i) (for i = 0...new-degree).
      (trace-set-subtrace
          old-trace
          "underlying-curve"
          (reduce
            (fn [tr i] (trace-set-value tr `("coeffs" ~i) (nth new-coeffs i)))
            {"degree" {:value new-degree}}
            (range (+ 1 new-degree)))))))

<a name="323"></a>
<h3 style="color:blue">Problem 3.2.3: Applying asymmetric MH proposals</h3>

Now that we've written some asymmetric proposers, let's write the library function that will let us apply them. Like the symmetric proposer helper function, `custom-proposer-mh-step` accepts a model and a proposer, and returns a trace-to-trace function that includes logic for accepting and rejecting. Read the following code and fill in the `'...` with an expression that computes the log acceptance ratio $\log \alpha = \log \frac{p(t')}{p(t)} \frac{q(t \leftarrow t')}{q(t' \leftarrow t)}$:

In [ ]:
;; PROBLEM

;; The proposal is a generative function that:
;;  - accepts a current trace
;;  - returns a new (proposed) trace
;; such that
;;  - if a value appears in the new trace that was not in the old trace,
;;    the value was _sampled_ by proposal at the same address as in the model

(defn custom-proposer-mh-step [model proposer]
  (fn [current-trace]
    (let [[_ _ current-trace-score]               ;; Evaluate log p(t)
          (infer-and-score :procedure model
                           :observation-trace current-trace)
        
          [proposed-trace all-proposer-choices _] ;; Sample t' ~ q(• <- t)
          (infer-and-score :procedure proposer
                           :inputs [current-trace])
                
          [_ _ new-trace-score]                   ;; Evaluate log p(t')
          (infer-and-score :procedure model
                           :observation-trace proposed-trace)
        
          [_ _ forward-proposal-score]            ;; Estimate log q(t' <- t)
          (infer-and-score :procedure proposer
                           :inputs [current-trace]
                           :observation-trace all-proposer-choices)
        
          [_ _ backward-proposal-score]          ;; Estimate log q(t <- t')
          (infer-and-score :procedure proposer
                           :inputs [proposed-trace]
                           :observation-trace proposed-trace)
        
          log-acceptance-ratio                  ;; Compute estimate of log [p(t')q(t <- t') / p(t)q(t' <- t)]
          '...]
      
        (if (flip (exp log-acceptance-ratio))   ;; Decide whether to accept or reject
            proposed-trace
            current-trace))))

In [ ]:
;; PROBLEM

;; The proposal is a generative function that:
;;  - accepts a current trace
;;  - returns a new (proposed) trace
;; such that
;;  - if a value appears in the new trace that was not in the old trace,
;;    the value was _sampled_ by proposal at the same address as in the model

(defn custom-proposer-mh-step [model proposer]
  (fn [current-trace]
    (let [[_ _ current-trace-score]               ;; Evaluate log p(t)
          (infer-and-score :procedure model
                           :observation-trace current-trace)
        
          [proposed-trace all-proposer-choices _] ;; Sample t' ~ q(• <- t)
          (infer-and-score :procedure proposer
                           :inputs [current-trace])
                
          [_ _ new-trace-score]                   ;; Evaluate log p(t')
          (infer-and-score :procedure model
                           :observation-trace proposed-trace)
        
          [_ _ forward-proposal-score]            ;; Estimate log q(t' <- t)
          (infer-and-score :procedure proposer
                           :inputs [current-trace]
                           :observation-trace all-proposer-choices)
        
          [_ _ backward-proposal-score]          ;; Estimate log q(t <- t')
          (infer-and-score :procedure proposer
                           :inputs [proposed-trace]
                           :observation-trace proposed-trace)
        
          log-acceptance-ratio                  ;; Compute estimate of log [p(t')q(t <- t') / p(t)q(t' <- t)]
          (- (+ new-trace-score backward-proposal-score)
             (+ current-trace-score forward-proposal-score))]
      
        (if (flip (exp log-acceptance-ratio))   ;; Decide whether to accept or reject
            proposed-trace
            current-trace))))

**Discuss here: resimulation MH. Why we can't do it yet.**

It's time to put our algorithm to the test on some more interesting datasets!

Let's create a couple datasets, with polynomials of varying degrees, with and without outliers:

In [ ]:
;; Create some fake datasets for curve-fitting
(def point-count 10)

(def x-min -5)
(def x-max 5)
(def x-range (- x-max x-min))
(def x-interval (/ x-range (- point-count 1)))

(def xs (map-indexed (gen [i interval] (+ -5. (* interval i))) (repeat point-count x-interval)))

(def ys-linear (map #(+ (* 2 %) 1) xs))

(defn add-outlier [ys]
    (let [idx (uniform-discrete (range (count ys)))]
      (map-indexed (fn [i y] (if (= i idx) (gaussian 0 10) y)) ys)))

(def ys-linear-outlier (add-outlier (add-outlier ys-linear)))

(def ys-quadratic (add-outlier (map #(gaussian (- (* % %) %) 0.5) xs)))

We now write a function `inference-step` that composes all the proposers we've written so far into a single update, which we can then iterate many times:

In [ ]:
(defn address-contains? [addr elem]
    (some #{elem} addr))

(defn inference-step [xs ys tr]
  (let [model
        (with-args curve-model [xs])
        
        curve-step
        (custom-proposer-mh-step model curve-proposer)
        
        noise-step
        (custom-proposer-mh-step model noise-proposer)
        
        outlier-steps
        (map (fn [i] (symmetric-proposer-mh-step model (make-outlier-proposer i))) (range (count xs)))
        
        coeff-addresses
        (filter (fn [addr] (address-contains? addr "coeffs")) (addresses-of tr))
        
        coeffs-step
        (symmetric-proposer-mh-step model (make-gaussian-drift-proposer coeff-addresses 0.1))
        
        full-step
        (reduce comp identity `[~curve-step ~coeffs-step ~noise-step ~coeffs-step ~@outlier-steps])]
      
      (full-step tr)))

Here's the helper function that runs our MH algorithm for many iterations:

In [ ]:
(defn run-mh-curve-fitting
  [xs ys n-iters]
  (let [[_ initial-trace _]
        (infer-and-score :procedure curve-model :inputs [xs] :observation-trace (make-observation-trace ys))
        
        inferred-trace
        (reduce (fn [tr i] (inference-step xs ys tr))
            initial-trace (range n-iters))
        
        [_ _ score]
        (infer-and-score :procedure curve-model :inputs [xs] :observation-trace inferred-trace)]
      
      (pprint inferred-trace)
      (println (str "Final score: " score))))

And here are the results on each dataset:

In [ ]:
(run-mh-curve-fitting xs ys-linear 500)

In [ ]:
(run-mh-curve-fitting xs ys-linear-outlier 500)

In [ ]:
(run-mh-curve-fitting xs ys-quadratic 500)

## 3.3) Gibbs sampling

Finally, we add one more function to our library: `make-gibbs-step`. A "Gibbs sampling step" is a proposal that updates a single variable in our trace according to the _actual_ conditional distribution of that variable, given the current values of all other variables in the trace. In this case, denoting the updated variable as $v$ and the rest of the variables as $t_{\text{other}}$, we have:

\begin{align*}
\alpha =& \frac{p(t')}{p(t)} \frac{q(t \leftarrow t')}{q(t' \leftarrow t)}\\
=& \frac{p(t'_{\text{other}})p(v'|t'_{\text{other}})}{p(t_{\text{other}})p(v \mid t_{\text{other}})} \frac{p(v \mid t_{\text{other}})}{p(v' \mid t'_{\text{other}})}\\
=& \frac{p(t'_{\text{other}})}{p(t_{\text{other}})} = 1\\
\end{align*}

where the last equality follows because the proposal $t'$ only updates $v$ and leaves the rest of the trace unchanged.

In general, $p(v \mid t_{\text{other}})$ is not easy to compute. But for finite, discrete random variables $v$ that don't affect control flow in our trace, we can use the following strategy:

1. For each possible value $u$ that $v$ could take on, consider a trace $t_{v:=u}$ in which $v$ has been set to $u$, and get its probability $p_u := p(t_{v:=u})$.
2. Renormalize the $p_u$ values and sample from the normalized distribution, which is exactly $p(v|t_{\text{other}})$.

<a name="331"></a>
<h3 style="color:blue">Problem 3.3.1: Enumerative Gibbs sampling</h3>

Finish the implementation of `make-gibbs-step`, which takes in a model, an address, and the support of the random variable at that address (a list of possible values it may take on).

The missing piece below (`'...`) is a function for computing $\log p(t_{v:=u})$: the log probability of the trace that has the variable in question set to a certain value $u$. The function `log-categorical` takes care of normalization for us.

In [ ]:
;; PROBLEM

(defn make-gibbs-step
    [model address support]
  (fn [current-trace]
    (let [log-scores
          (map (fn [u] '...) support)]
      (trace-set-value 
         current-trace 
         address 
         (nth support (log-categorical log-scores))))))

In [ ]:
;; SOLUTION

(defn make-gibbs-step
    [model address support]
  (fn [current-trace]
    (let [log-scores 
          (map (fn [value] 
                   (nth (infer-and-score :procedure model
                                         :observation-trace
                                         (trace-set-value current-trace address value)) 2))
                support)]
        (trace-set-value 
            current-trace 
            address 
            (nth support (log-categorical log-scores))))))

In [ ]:
;; TEST
((make-gibbs-step (with-args curve-model [[0]])
                      '("data" 0 "is-outlier?") 
                      [true false])
 
 {"underlying-curve" {"degree" {:value 0}
                      "coeffs" {0 {:value 0}}}
  "add-noise" {"prob-outlier" {:value 0.1}
               "noise-level" {:value 0.3}}
  "data" {0 {"is-outlier?" {:value true}
             "y" {:value 0}}}})

<a name="4"></a>
# 4) Metaprob generative functions: the full picture

---

**Goals of this section**
- We begin by revisiting importance sampling, and justifying it mathematically. Usually, we think of importance sampling as working with an actual distribution `p` and a proposal distribution `q`.
- How would we add support for importance sampling with a _custom_ proposal? The custom proposal could be "data-driven": we pass in an observation trace, and get out a proposal that is a generative function, which traces at the non-observed addresses. We write this procedure, and look at a couple examples, including `add-errors`.
- We make a couple remarks:
    - The "custom proposal" interface seems somewhat similar to what `infer-and-score` already does: it takes in a set of observations, and then "fills in" the missing pieces. Introduce `make-constrained-generator`, as a default "custom proposal." Write infer-and-score in terms of it.
    - In general, a proposal may make _more_ choices than the model does. We handle this by requiring that a "constrained generator" _return_ the trace it generates.
    - We can give a generative function a _default_ custom proposal by using a custom `make-constrained-generator`. Because the default `make-constrained-generator` calls children recursively, this allows our proposal to be used compositionally. (Consider the code for `spelling-model`: it _calls_ `add-errors`.)
- Let's return to MCMC. Recall our proposals for the noise and curve parameters. _They are doing exactly what `infer-and-score` would do if given an observation trace of `current-trace` minus the parameters to resimulate._ This motivates a way of writing a "resimulation MH" helper function. It automatically uses whatever fancy internal proposal is attached.
---

<a name="41"></a>
## 4.1) Importance sampling with a custom proposal

We have now seen that simple score-based inference algorithms like likelihood weighting are sufficient for answering questions about our coin model, and that iterative inference algorithms are sufficient for answering questions about our curve-fitting model. We will now augment our library with algorithms capable of handling our last example: spelling correction.

We begin by revisiting importance sampling, with a twist: we will now allow custom proposal distributions $q$. Instead of directly using the $\log \frac{p(t+u)}{q(t; u)}$ score returned by `infer-and-score`, we will allow the user to provide their own $q(t; u)$, and compute the ratio automatically. The idea of a custom $q(t; u)$ should be familiar from the Gen problem set, where you implemented "bottom-up" or "data-driven" proposals for certain problems.

<a name="411"></a>
<h3 style="color:blue">Problem 4.1.1: Importance resampling with a custom proposal</h3>




Our _importance resampling with a custom proposal_ works as follows. It accepts as input:
- a model.
- a function `make-proposer`, which accepts an _observation trace_ $u$ and returns a "constrained proposer", which is a generative function that has the same arguments as the model, but that makes random choices only at non-observed addresses, essentially sampling $t$ so that $t+u$ is a complete trace of the model.
- a list of inputs to the model (and proposer).
- an observation trace, $u$.
- $n$, a number of particles.

It then:
- samples $n$ proposed trace fragments $t_{1\cdots n}$ from the proposal, to create $n$ complete traces $t_i+u$ (where $+$ is `trace-merge`).
- computes scores $s_i = \log \frac{p(t_i+u)}{q(t_i; u)}$ for each of the $n$ particles.
- uses `log-categorical` to sample one of those particles according to the scores.

In [ ]:
;; PROBLEM
(defn importance-resampling-custom-proposal
    [model make-proposer inputs observations n]
  (let [proposed-traces '... ;; Each of the n proposed traces is generated by tracing the custom proposal, and merging the resulting trace with observations
        scores          '...] ;; The score of a trace t is log p(t)/q(t), i.e., log p(t) - log q(t). Use infer-and-score to get log scores under p and q.
    (nth proposed-traces (log-categorical scores))))

In [ ]:
;; SOLUTION
(defn importance-resampling-custom-proposal
    [model make-proposer inputs observations n]
  (let [custom-proposal 
        (make-proposer observations)
        
        proposed-traces
        (replicate n 
          (fn [] 
            (let [[_ t _]
                  (infer-and-score :procedure custom-proposal
                                   :inputs inputs)]
              (trace-merge t observations))))
        
        scores
        (map (fn [tr]
               (- (nth (infer-and-score :procedure model :inputs inputs :observation-trace tr) 2)
                  (nth (infer-and-score :procedure custom-proposal :inputs inputs :observation-trace tr) 2)))
             proposed-traces)]
    
    (nth proposed-traces (log-categorical scores))))

**TODO: Add simple Gaussian example with graphs?**

<a name="412"></a>
<h3 style="color:blue">Problem 4.1.2: A custom proposal for add-errors</h3>

Using custom proposals, we can build arbitrary heuristics into our inference algorithms. In this problem, you'll write some helper functions we'll use to define a custom proposal for `add-errors`.

**Framing**

Recall that `add-errors` works as follows:
1. Accept as input a "true string," e.g., "seattle"
2. Initialize current-string := true-string, and repeatedly:
    - Decide whether to introduce another typo. If not, just return the current-string.
    - If we are adding a typo, decide what kind (substitution, insertion, or deletion).
    - Choose the parameters (error index, new letter) for the typo.
    - Set current-string to the new typo'd string, and repeat.

Our custom proposal will work the same way, except it will _already know the observed final string_. Therefore, it can be "smart" when sampling. If it knows that the observed string was "sattle" and the current string is "seattle," our custom proposal can:
- Force the "done-adding-typos?" choice to be false.
- Decide to add a deletion.
- Decide that the deletion will be at index 1.
- Once the current string is "sattle", with very high probability, stop introducing typos.

Proposal distributions are like "guessers": we see the observed string, and we want to guess the path of typos that was made to get there — the proposal code literally "acts out" the guess, simulating a sequence of typos that could have led from the original string to the observed string. We are writing "smart guessers," that use simple logical heuristics to make better guesses, or to "guide" them while they simulate. You'll now write some helper functions that implement this sort of logic.

**Part 1: Deciding whether to add a typo.**

Write a Clojure function that accepts a current string and an observed string, and determines whether or not we should guess that another typo was made.

In [ ]:
;; Problem 4.1.2, part 1: decide whether to add another typo
(defn should-probably-add-typo? 
    [current-string observed-string]
    '...)

**Part 2: Deciding what kind of typo to add**

Write a Clojure function that returns a preferred type of error to introduce (`:insertion`, `:deletion`, or `:substitution`) given the current string and the observed string.

In [ ]:
;; Should return :substitution, :insertion, or :deletion
;; based on (e.g.) lengths of the the two strings.
(defn preferred-error-type
    [current-string observed-string]
    '...)

**Part 3: Checking whether an index is a good place to introduce an error**

Write a Clojure function that evaluates whether an index `i` might be an especially promising place to introduce an error. In Clojure, `subs` is used to get substrings: `(subs i (+ i 1))` gets the ith letter of a string. Note that `i` may be anywhere in the range `[0...(count current-string)]` inclusive; there are no guarantees about how it relates to the length of the _observed_ string.

In [ ]:
;; Determine whether `i` is a good index at which to commit an error
;; (insertion, substitution, or deletion).
;; Use (subs i (+ i 1)) to get the ith letter of a string.
;; `i` is guaranteed to be in the range [0...(count current-string)] inclusive.
(defn good-index-for-error?
  [i current-string observed-string]
  '...)

**Part 4: Deciding which new letter to introduce**

We provide code for the final heuristic, which computes a probability distribution over letters that we might want to introduce into the string, assuming we have chosen to make a substitution or insertion. For each possible letter `c` we might want to introduce, it computes the minimum distance of `c` to the chosen index `error-index` in the observed string, and makes letters exponentially less likely as this distance increases.

In [ ]:
(defn distance-from-index
  [s idx c]
  (min
    (+ 1 (or (index-of (clojure.string/reverse (subs s 0 (min (count s) idx))) c) (count s)))
         (or (index-of (subs s (min (count s) idx)) c) (+ (count s) 1))))

(defn probability-of-letter 
    [l observed-string error-index]
    (expt 0.01 (distance-from-index observed-string error-index l)))

(defn letter-probabilities
    [observed-string error-index]
    (zipmap
      alphabet
      (map #(probability-of-letter % observed-string error-index) alphabet)))

**Exercise solutions**

In [ ]:
;; Answer key

(def should-probably-add-typo? not=)
(defn preferred-error-type
  [current-string observed-string]
  (cond
    (< (count current-string) (count observed-string)) :insertion
    (> (count current-string) (count observed-string)) :deletion
    (= (count current-string) (count observed-string)) :substitution))


(defn good-index-for-error?
  [i current-string observed-string]
  (or (>= i (count observed-string))
      (<= (count current-string) i)
      (not= (subs current-string i (+ i 1)) (subs observed-string i (+ i 1)))))

**Putting it all together**

We now write our "smart proposer" for `add-errors`. It accepts an observation trace, and pulls out the observed string in a variable `observed-string`. It then returns a generative function that acts like `add-errors`, but smarter:

1. Whenever we need to decide whether we're `done-adding-errors?`, we call `should-probably-add-typo?`. If the answer is yes, we give ourselves zero probability of being done adding errors; otherwise, with 99.9% probability, we _do_ finish up, and return the final string. This is an important point: in importance sampling, it should be _possible_ for our proposal distribution to guess _any_ valid trajectory: this is why, even if we don't need to add more typos, we still give ourselves a 0.1% chance of adding more — under our model, someone might delete an `e` and add it back in, introducing two typos despite returning the string as is.

2. If we _do_ decide to add a typo, instead of calling `add-error`, we call a "smart version," which we define below.

In [ ]:
;; Forward declaration for 
(declare make-smart-add-error)

;; We begin by writing make-add-errors-proposer, which takes in
;; an observation trace
(defn make-add-errors-proposer
  [observation-trace]
  (let [observed-string (trace-value observation-trace "final-string")
        smart-add-error (make-smart-add-error observed-string)]
    
    ;; This is the "smart add-errors"
    (gen {:tracing-with t} 
      [true-string]
      (let [helper
            (fn [current-string i]
              (if (t `(~i "done-adding-errors?") 
                  flip
                  [(if (should-probably-add-typo? current-string observed-string) 
                      0.000
                      0.999)])
               ;; If we are done adding errors, just return the current string
               (t "final-string" exactly [current-string])
            
               ;; Otherwise, add another typo and loop
               (recur (t i smart-add-error [current-string]) 
                      (+ i 1))))]
          
          ;; Start the process going
          (helper true-string 0)))))

Most of the interesting stuff happens in `make-smart-add-error`, below, which creates a "smart version" of `add-error` based on a particular observed string. This smart version will try to add an error to the `current-string` that makes it closer to the observed string.

Just like the model, we add an error in three steps: choosing a type of error, an index for the error, and potentially a new letter to introduce.

1. We use the helper function you wrote, `preferred-error-type`, to figure out what kind of error is most likely to help us. We sample a categorical random variable `"error-type"` that assigns 20 times more weight to the preferred error type than to the other two. Note that we are being careful to trace our choices at the same address as the model did.

2. We do the same thing for the indices: for each possible error index, we call `good-index-for-error?` (which you wrote above) to determine whether to give this index a relative weight of 1 or 0.02 in our categorical draw.

3. Finally, we do the same thing to choose a letter, if necessary. We then apply the error.

In [ ]:
(defn score-error-type
  [error-type current-string observed-string]
  (if (= error-type (preferred-error-type current-string observed-string))
      1 0.05))

(defn make-smart-add-error 
    [observed-string]
  (gen {:tracing-with t} 
    [current-string]
    (let [;; Error type
          error-types
          [:insertion :deletion :substitution]
             
          error-type-probabilities
          (zipmap 
            error-types
            (map #(score-error-type % current-string observed-string) error-types))
             
          error-type
          (t "error-type" categorical [error-type-probabilities])
          
          
          ;; Error index
          error-index-upper-bound
          (+ (count current-string) (if (= error-type :insertion) 1 0))
          
          index-probs
          (map
            (fn [i] (if (good-index-for-error? i current-string observed-string) 1 0.02))
            (range error-index-upper-bound))
          
          error-index
          (t "error-index" categorical [index-probs])
          
          ;; Letter to introduce
          letter-probs
          (letter-probabilities observed-string error-index)
          
          new-letter
          (if (= error-type :deletion) "" (t "new-letter" categorical [letter-probs]))]
         
         ;; Apply the typo
         (cond 
           (= error-type :deletion) (delete-at-index current-string error-index)
           (= error-type :insertion) (insert-at-index current-string error-index new-letter)
           (= error-type :substitution) (replace-at-index current-string error-index new-letter)))))

Let's understand what this proposal is doing:

In [ ]:
;; Ask our smart add error to simulate a typo, starting at "computer", if we
;; know that the final string (after potentially multiple typos) is "compater"
(let [[_ t _] (infer-and-score :procedure (make-smart-add-error "compater")
                 :inputs ["computer"])]
    (plot-trace t 200))

In [ ]:
;; Ask our smart add error to simulate a typo, starting at "computer", if we
;; know that the final string (after potentially multiple typos) is "cmptr"
(let [[_ t _] (infer-and-score :procedure (make-smart-add-error "cmptr")
                 :inputs ["computer"])]
    (plot-trace t 200))

We can use the smart _add-errors_ proposal to simulate a whole stream of typos that bring us from an intial string like "computer" to a final string like "cmptr". Note that our heuristics don't always give us the shortest path:

In [ ]:
;; Ask our smart add error to simulate a typo, starting at "computer", if we
;; know that the final string (after potentially multiple typos) is "cmptr"
(let [[_ t _] (infer-and-score :procedure (make-add-errors-proposer 
                                              {"final-string" {:value "cmptr"}})
                 :inputs ["computer"])]
    (pprint t))

**On its own, just running this proposer is _not_ doing Bayesian inference.** The point is that we can pass it to our new importance resampling procedure:

In [ ]:
(pprint
  (importance-resampling-custom-proposal
      add-errors
      make-add-errors-proposer
      ["computer"]
      {"final-string" {:value "cmptr"}}
      100))

This almost always _does_ give us the shortest path -- because the shortest path is much more likely, under our posterior, than the various longer paths that our proposer sampled directly. Running the cell above multiple times, we also see some variation in the _order_ of the deletions: none is more likely under our model than another, so the various deletion orders should appear with relatively equal frequency.

We still have not solved our original problem: our full `spelling-model` is not just `add-errors`, but also includes a choice of city. We'd like some way to say: "do importance resampling for `spelling-errors`, but when guessing the typo sequence, defer to `add-errors`." If we could do this, importance resampling would _repeatedly_:

- Pick a city at random.
- Find a way (using our smart proposal) to get from that city name to the observed string.
- Score the entire trajectory according to $\frac{p(t+u)}{q(t; u)}$. Note that when the string of typos is long, $p(t+u)$ will be very small, whereas $q(t; u)$ will not: $q$ is  quite _likely_ to generate long strings of typos, as its `done-adding-errors?` flip is often false with probability 1. So scores involving longer typo sequences will be much smaller than scores involving shorter ones.

Then it would pick one of those traces at random based on the scores. Trajectories with true cities at shorter edit distances from the observed string would be much more likely to be chosen.

We could implement this now, by writing a custom proposer for `spelling-model`, which calls out to the custom proposer for `add-errors`. But this seems redundant: is there a way that we can have this behavior happen _automatically_, by somehow _attaching_ our custom proposal to `add-errors` itself? Ideally, _every time_ we wanted to do a constrained run of `add-errors`, even within some other model, our smart proposal would be called. This is possible, but to see how, we'll need to take a closer look at the Metaprob generative funciton interface!

<a name="42"></a>
## 4.2) The `make-constrained-generator` method

The `make-proposer` function that `importance-resampling-custom-proposal` accepts has certain similarities to `infer-and-score` (or at least, the `infer` part): it takes in an observation trace, and makes choices to "fill in" the rest of the trace.

In fact, this relationship is even closer than one might first suspect. `infer-and-score` is implemented in terms of a more basic operation on Metaprob generative functions: `make-constrained-generator`.

The `make-constrained-generator` function takes in a generative function and an observation trace, and returns a version of the generative function that 
1. samples _only_ at addresses _not_ present in the observation trace.
2. returns the `infer-and-score` return value -- a vector of a model return value, a full "filled in" trace, and a score.

Let's take a closer look:

In [ ]:
;; We make a version of curve-model that is constrained to sample degree 1
(def line-model
  (make-constrained-generator curve-model {"underlying-curve" {"degree" {:value 1}}}))

;; When we run line-model, it returns ys, a complete curve-model trace, and a score
(pprint (line-model [0 1 2]))

In [ ]:
;; But we can also _trace_ line-model:

(let [[_ t _] (infer-and-score :procedure line-model
                               :inputs [[0 1 2]])]
    (plot-trace t 500))


Note that this trace _does not_ contain `degree`. This is the trace _of the constrained generator itself_, **not** the trace of `curve-model` _returned_ by the constrained generator.

Because `make-constrained-generator` accepts an observation trace and returns a proposer that traces at the same addresses as the original function, it would work as a "custom proposer" argument to our importance resampling procedure:

In [ ]:
(pprint
  (importance-resampling-custom-proposal
    curve-model
    (fn [observations] (make-constrained-generator curve-model observations))
    [[0 1 2]]
    (make-observation-trace [1 2 3])
    10000))

;; Equivalent to just using `importance-resample` from section 2, with no custom
;; proposal. The only difference is that the `importance-resample` from section 2
;; only calls `infer-and-score` _once_: 

**So, what is a generative function?**

A Metaprob generative function is actually just an object with two methods: `run-in-clojure` and `make-constrained-generator`. The `infer-and-score` function we saw earlier is implemented, in just a couple lines of code, in terms of `make-constrained-generator`: 

In [ ]:
(pprint (get infer-and-score :generative-source))

(Note that it is a generative function, which is itself traceable: when we trace infer-and-score, we see that it makes random choices for all addresses that were not in the observation trace.)

The `gen` macro automatically synthesizes implementations of the two methods based on source code, but we can also specify them manually. That is, if we can implement by hand the following two functions:

1. `run-in-clojure`: a Clojure function that accepts some arguments and produces a return value.
2. `make-constrained-generator`: a Clojure function that accepts an observation trace, and returns _another generative function_ that:
    1. accepts the same arguments as `run-in-clojure`,
    2. makes choices at the model addresses not covered by the observation trace (and potentially, some of its own random choices), and
    3. returns a _model return value_, a _complete model trace_ (consisting of observations plus any new random choices it "filled in"), and a _score_ $\log \frac{p(t+u)}{q(t; u)}$,
    
    
then we can manually construct a generative function, using Metaprob's `make-generative-function` procedure. In fact, the `gen` macro desugars to a call to `make-generative-function`, with the proper implementations of each of the two above functions.

In [ ]:
;; Somewhat hard-to-read code generated by the `gen` macro.
;; Notice that it calls `metaprob.syntax/make-generative-function`, passing in
;; an implementation of `run-in-clojure` and of `make-constrained-generator`.
;; (You are not supposed to understand these implementations!)
(pprint (macroexpand '(gen {:tracing-with t} [] (t '() flip [0.5]))))

This suggests that we could create a generative function `add-errors` that had a default `make-constrained-generator` which relied on our smart proposer! This is what we will do in the next subsection.

<a name="43"></a>
## 4.3) Custom generative functions

We can make custom generative functions with `make-generative-function`:

```clojure
(make-generative-function
    run-in-clojure
    make-constrained-generator)
```

We'd like to do something like this:

```clojure
(def new-add-errors
  (make-generative-function
      ;; To run in Clojure, just run the old add-errors
      add-errors
      ;; To make-constrained-generator, use our special method
      make-add-errors-proposer))
```

There are two problems with this: 
1. The `make-add-errors-proposer` does not do everything that `make-constrained-generator` needs to do: it doesn't return a vector of `[retval full-trace score]`. We had relied on the `importance-resampling-custom-proposal` function to compute scores _for_ us, as well as to compute the full trace by merging observations in.
2. A `make-constrained-generator` should be able to handle _any_ observation trace -- including empty ones! Our `make-add-errors-proposer` function assumes that the final string has been observed.

We can write a helper function, `with-custom-proposal-attached`, that takes in an original generative function (like `add-errors`), a new custom proposal (like `make-add-errors-proposer`), and a _condition for use_ of the custom proposer: a predicate on observation traces that determines whether this proposer makes sense to use in this context. It returns a generative function that behaves as follows:

1. Its `run-in-clojure` is just the original generative function.
2. Its `make-constrained-generator` takes in an observation trace, and:
    1. Runs `condition-for-use` on the observation trace to determine whether our custom proposer is applicable.
    2. If it _is not_ applicable, just return `(make-constrained-generator orig-generative-function observation-trace)`, to pass the buck to the original proposing behavior.
    3. If it _is_ applicable, return a generative function that takes in model arguments, and:
        1. Traces the custom proposer on the current observations and model arguments.
        2. Calculates p(proposed trace + observations) and q(proposed trace; observations)
        3. Merges the proposed trace and the observations to get a complete model trace
        4. Returns `[retval full-trace score]`.

In [ ]:
(defn with-custom-proposal-attached
  [orig-generative-function make-custom-proposer condition-for-use]
  (make-generative-function
    ;; To run in Clojure, use the same method as before:
    orig-generative-function
      
    ;; To create a constrained generator, first check if 
    ;; the condition for using the custom proposal holds.
    ;; If so, use it and score it.
    ;; Otherwise, use the original make-constrained-generator
    ;; implementation.
    (fn [observations]
      (if (condition-for-use observations)
        (gen {:tracing-with t} [& args]
          (let [custom-proposal 
                (make-custom-proposer observations)
                
                [_ tr _] 
                (t '() infer-and-score 
                   [:procedure custom-proposal,
                    :inputs args])
                
                proposed-trace
                (trace-merge observations tr)
                
                [v tr2 p-score]
                (infer-and-score :procedure orig-generative-function
                                 :inputs args
                                 :observation-trace proposed-trace)
                
                [_ _ q-score]
                (infer-and-score :procedure custom-proposal
                                 :inputs args
                                 :observation-trace proposed-trace)]
              [v proposed-trace (- p-score q-score)]))
          (make-constrained-generator orig-generative-function observations)))))

We can now create the new `add-errors` that we wanted:

In [ ]:
(def add-errors
    (with-custom-proposal-attached
        add-errors
        make-add-errors-proposer
        (fn [tr] (trace-has-value? tr "final-string"))))

Built-in functions like `infer-and-score` now automatically make use of our "smart" behavior:

In [ ]:
(pprint 
  (infer-and-score :procedure add-errors
                 :inputs ["computer"]
                 :observation-trace {"final-string" {:value "cmptr"}}))

The same goes for the vanilla `importance-resample` routine we wrote in [Section 2.4](#24):

In [ ]:
(pprint 
    (importance-resample (with-args add-errors ["computer"])
                         {"final-string" {:value "cmptr"}}
                         100))

And if we re-run the cell that defines `spelling-model` so that it uses the new `add-errors`, then inference about `spelling-model` will also take advantage of the new logic:

In [ ]:
;; Same exact code from section 2:
(def spelling-model
  (gen {:tracing-with t} [] 
       (let [city (t "true-city" categorical [washington-cities])] 
           (t "add-errors" add-errors [city]))))

In [ ]:
;; Inferring that "seattle" was the true string
(pprint 
    (importance-resample spelling-model
                         {"add-errors" {"final-string" {:value "satl"}}}
                         100))

It's worth taking a moment to understand why improving the `add-errors` proposal led to accurate inference in the `spelling-model` problem:

- The `spelling-model` proposal first guesses a "true city," then guesses a sequence of typos.
- If it can't guess a sequence of typos that leads to the actual observed string, it assumes it must have been wrong about the true city. (More specifically, this particular particle gets a score of $\frac{p}{q} = 0$, because the sampled trace has probability 0 under our model $p$. As such, it has no chance of being sampled in the resampling step.)
- Before we added a custom proposal, `add-errors` would almost never make guesses that led to the actual observed string. So usually, _none_ of the particles would have non-zero weight.
- _With_ the custom proposal, `add-errors` _always_ samples a possible sequence of typos from the chosen city to the observed string. The score, $\frac{p}{q}$, is generally lower for longer sequences of typos, because $p$ places relatively small probability mass on long sequences of typos, whereas $q$ is happy to keep making typos as long as necessary. As such, particles where the true city required fewer typos to get to the observed city tend to have higher weights.

Let's use `importance-resample` on `spelling-model` with observation `"spatkne"`, to estimate the posterior probability that it came from "seattle" vs. "spokane". Our posterior distribution factors in not just the relative typo probabilities, but the relative _city populations_ (recall that our model posits more populous cities as appearing more frequently).

In [ ]:
;; Getting a posterior distribution over "seattle" vs. "spokane" given
;; observed string "spatkne"
(let [results 
      (replicate 100 
        (fn [] (trace-value 
                 (importance-resample spelling-model
                   {"add-errors" {"final-string" {:value "spatkne"}}}
                   30)
                 "true-city")))]
  (str "Seattle: " (count (filter (fn [city] (= city "seattle")) results)) ", "
       "Spokane: " (count (filter (fn [city] (= city "spokane")) results))))

<a name="431"></a>
<h3 style="color:blue">Problem 4.3.1: Exact inference for subproblems in coin model</h3>


In this problem, we'll use `with-custom-proposal-attached` to create new, smarter versions of `coin-model-2` and `coin-model-3`, two of the helper functions used in `coin-model`. Recall that `coin-model-2` randomly samples a probability-of-heads $p$ from the set $\{0, 1\}$, whereas `coin-model-3` samples $p$ uniformly from the _interval_ $[0, 1]$. Each then performs $n$ coin flips, traced at addresses $0, 1, \dots, n-1$.

For Coin Model 2:
- Based on the observations, how can you make a smart guess for `p`? Fill in the missing code.

For Coin Model 3:
- You may use this result from probability theory: If we have seen $a$ heads and $b$ tails, and our prior belief was that the probability of heads was distributed uniformly in $[0, 1]$, then the posterior $p(\texttt{p}\,|\,\text{observations}) = \text{Beta}(a+1, b+1)$.

In [ ]:
;; Problem: Write modified versions of coin-model-2 and coin-model-3 that have custom proposals attached.

;; Helper: extracts all observed coin flips from an observation trace, and returns them as a list.
(defn get-observed-flips [observation-trace]
  (map #(trace-value observation-trace %)
    (filter number? (map first (addresses-of observation-trace)))))


;; Replace 'probability-for-choosing-0... and 'probability-for-choosing-1... in
;; both clauses of the if expression to make this a good custom proposal for
;; coin-model-2:
(defn coin-model-2-make-custom-proposal
    [observation-trace]
    (let [flips (get-observed-flips observation-trace)]
      (gen {:tracing-with t} [n]
        (let [p (t "p"
                   categorical
                   [(if (empty? (filter true? flips))
                       ;; Replace each symbol 'probability-for-choosing-x... 
                       ;; with your own number between 0 and 1 (inclusive)
                       ['probability-for-choosing-0... 'probability-for-choosing-1...]
                       ['probability-for-choosing-0... 'probability-for-choosing-1...])])]
          
          ;; Flip the rest of the coins
          (map
            (fn [i] (if (trace-has-value? observation-trace i)
                        (trace-value observation-trace i)
                        (t i flip [p])))
              (range n))))))

;; Fill in the condition-for-use, a predicate on traces
(def modified-coin-model-2
    (with-custom-proposal-attached
        coin-model-2
        coin-model-2-make-custom-proposal
        ;; Only use the custom proposal when p is unknown
        '...))

;; Write a custom proposal maker for coin-model-3.
(defn coin-model-3-make-custom-proposal
    [observation-trace]
    (let [flips (get-observed-flips observation-trace)]
      (gen {:tracing-with t} [n]
        ;; Draw `p` from the exact posterior, (beta (+ n-heads 1) (+ n-tails 1)),
        ;; where n-heads and n-tails are the observed number of heads and tails.
        ;; Also make sure to flip any coins at addresses 0, ..., n-1 that are not
        ;; already in the observation trace.
        '...
        )))

;; Fill in the condition-for-use, a predicate on traces
(def modified-coin-model-3
    (with-custom-proposal-attached
        coin-model-3
        coin-model-3-make-custom-proposal
        ;; Only use the custom proposal when p is unknown
        '...))

;; Testing:

(let [[_ _ s-3]
      (infer-and-score :procedure modified-coin-model-3 :inputs [10]
                      :observation-trace {1 {:value true}, 2 {:value true}, 3 {:value true}})
      [_ tr2 s-2]
      (infer-and-score :procedure modified-coin-model-2 :inputs [10]
                      :observation-trace {1 {:value true}, 2 {:value true}, 3 {:value true}})]
    (and (< -1.39 s-3 -1.38)
        (every? true? (get-observed-flips tr2))
        (= s-2 (log 0.5))))

In [ ]:
;; Answer key:

(defn get-observed-flips [observation-trace]
  (map #(trace-value observation-trace %)
    (filter number? (map first (addresses-of observation-trace)))))


;; Replace 'probability-for-choosing-0... and 'probability-for-choosing-1... in
;; both clauses of the if expression to make this a good custom proposal for
;; coin-model-2:
(defn coin-model-2-make-custom-proposal
    [observation-trace]
    (let [flips (get-observed-flips observation-trace)]
      (gen {:tracing-with t} [n]
        (let [p (t "p"
                   categorical
                   [(if (empty? (filter true? flips))
                       
                       [1 0]
                       [0 1])])]
          
          ;; Flip the rest of the coins
          (map
            (fn [i] (if (trace-has-value? observation-trace i)
                        (trace-value observation-trace i)
                        (t i flip [p])))
              (range n))))))

;; Fill in the condition-for-use, a predicate on traces
(def modified-coin-model-2
    (with-custom-proposal-attached
        coin-model-2
        coin-model-2-make-custom-proposal
        ;; Only use the custom proposal when p is unknown
        (fn [tr] (not (trace-has-value? tr "p")))))

;; Write a custom proposal maker for coin-model-3.
(defn coin-model-3-make-custom-proposal
    [observation-trace]
    (let [flips (get-observed-flips observation-trace)]
      (gen {:tracing-with t} [n]
        ;; Draw `p` from the exact posterior, (beta (+ n-heads 1) (+ n-tails 1)),
        ;; where n-heads and n-tails are the observed number of heads and tails.
        ;; Also make sure to flip any coins at addresses 0, ..., n-1 that are not
        ;; already in the observation trace.
        (let [n-heads (count (filter true? flips))
              n-tails (count (filter false? flips))
              p (t "p" beta [(+ 1 n-heads) (+ 1 n-tails)])]
          
          ;; Flip the rest of the coins
          (map
            (fn [i] (if (trace-has-value? observation-trace i)
                        (trace-value observation-trace i)
                        (t i flip [p])))
              (range n))))))

;; Fill in the condition-for-use, a predicate on traces
(def modified-coin-model-3
    (with-custom-proposal-attached
        coin-model-3
        coin-model-3-make-custom-proposal
        ;; Only use the custom proposal when p is unknown
        (fn [tr] (not (trace-has-value? tr "p")))))

(let [[_ _ s-3]
      (infer-and-score :procedure modified-coin-model-3 :inputs [10]
                      :observation-trace {1 {:value true}, 2 {:value true}, 3 {:value true}})
      [_ tr2 s-2]
      (infer-and-score :procedure modified-coin-model-2 :inputs [10]
                      :observation-trace {1 {:value true}, 2 {:value true}, 3 {:value true}})]
    (and (< -1.39 s-3 -1.38)
        (every? true? (get-observed-flips tr2))
        (= s-2 (log 0.5))))

With this exact inference procedure, we can now automatically reap the benefits in our hypothesis-testing coin model. Here, the benefits are that we can get good results with fewer particles, and that there is less variance in our estimates. 

In [ ]:
(def orig-coin-model (gen {:tracing-with t} [n]
                          (let [which-model (t "which-model" uniform-discrete [[1 2 3]])
                                coin (nth [coin-model-1 coin-model-2 coin-model-3]
                                          (- which-model 1))]
                              (t '() coin [n]))))

(def new-coin-model (gen {:tracing-with t} [n]
                          (let [which-model (t "which-model" uniform-discrete [[1 2 3]])
                                coin (nth [coin-model-1 modified-coin-model-2 modified-coin-model-3]
                                          (- which-model 1))]
                              (t '() coin [n]))))

In [ ]:
;; Suppose we've seen the coin come up heads 100 times. 
;; What is the probability that Hypothesis 3 (uniform sample of p)
;; was correct?

;; The analytic answer is 1.9%. Compare the variance when using our original model
;; vs. the new one.

(pprint {"Original model results: " 
        (replicate 10 #(estimate-probability-is (with-args orig-coin-model [100])
                                                (fn [tr] (= 3 (trace-value tr "which-model")))
                                                (make-coin-observation-trace (repeat 100 true))
                                                50))
         
        "New model results: "
        (replicate 10 #(estimate-probability-is (with-args new-coin-model [100])
                                                (fn [tr] (= 3 (trace-value tr "which-model")))
                                                (make-coin-observation-trace (repeat 100 true))
                                                50))})

In [ ]:
;; We can calculate sample mean and variance for 200 runs with different numbers of particles 
;; to see the pattern more clearly:

(defn evaluate-coin-model-performance
  [model n-particles]
  (let [samples 
        (replicate 200
          #(estimate-probability-is
             (with-args model [100])
             (fn [tr] (= 3 (trace-value tr "which-model")))
             (make-coin-observation-trace (repeat 100 true))
             n-particles))
      
      sample-mean
      (/ (reduce + samples) (count samples))
      
      sample-variance
      (/ (reduce + (map #(expt (- % sample-mean) 2) samples)) (count samples))]
    
      [sample-mean sample-variance]))

In [ ]:
;; With few particles (original model), the estimate is way off, and variance is high (.2)
(evaluate-coin-model-performance coin-model 5)

In [ ]:
;; Increasing the number of particles leads to improvements:
(evaluate-coin-model-performance coin-model 10)

In [ ]:
(evaluate-coin-model-performance coin-model 20)

In [ ]:
(evaluate-coin-model-performance coin-model 30)

In [ ]:
;; With 50 particles, the estimate is quite good, but there is still a bit of variance
(evaluate-coin-model-performance coin-model 50)

In [ ]:
(evaluate-coin-model-performance coin-model 100)

In [ ]:
;; In the new coin model, with five particles, we have half as much variance as in the old
(evaluate-coin-model-performance new-coin-model 5)

In [ ]:
;; And our estimate improves quickly.
(evaluate-coin-model-performance new-coin-model 10)

In [ ]:
(evaluate-coin-model-performance new-coin-model 20)

In [ ]:
(evaluate-coin-model-performance new-coin-model 30)

In [ ]:
;; At 50 particles, we have lower variance than the original model
(evaluate-coin-model-performance new-coin-model 50)

In [ ]:
(evaluate-coin-model-performance new-coin-model 100)

<a name="44"></a>
## 4.4) Resimulation MH

One function in Gen's standard MCMC library that we didn't write in [Section 3](#3) is Resimulation MH: a resimulation proposal is a type of asymmetric proposal that resamples certain choices from the prior (the model), leaving the other choices fixed. (In fact, the `noise-proposer` and `curve-proposer` proposals that you wrote in [Section 3.2](#32) _were_ in fact resimulation proposals, but you had to write them by hand.)

Using `make-constrained-generator`, we can write a resimulation proposal in a more general way. Here is another way we could have written `noise-proposer`, which proposes a new trace by resampling the `prob-outlier` and `noise-level` choices in `curve-model`:

In [ ]:
(def new-noise-proposal
  (gen {:tracing-with t} [old-trace]
    (let [addresses-to-propose
          (filter (fn [addr] (address-contains? addr "add-noise"))
                  (addresses-of old-trace))
          
          ;; Use `partition-trace` to split the old trace into
          ;; two pieces: a trace containing the old noise parameters
          ;; (which we ignore), and a trace containing the old values
          ;; of all other random variables (which we keep and call
          ;; "fixed-choices": these choices are fixed, i.e., won't be updated
          ;; by this proposal.)
          [_ fixed-choices] 
          (partition-trace old-trace addresses-to-propose)
          
          ;; Obtain a constrained generator, which works
          ;; like the original curve-model, but doesn't sample
          ;; choices that are already constrained by fixed-choices
          constrained-generator 
          (make-constrained-generator curve-model fixed-choices)
          
          ;; Run the constrained generator, tracing the choices
          ;; it makes (of new prob-outlier and noise-level). Get 
          ;; the complete proposped trace so that we can return it.
          [_ proposed-trace _] 
          (t '() constrained-generator [xs])]
      
      ;; Return the proposed trace
      proposed-trace)))

Note that:
- We filter the addresses we're interested in (noise-level and prob-outlier) by searching for all addresses that are under the `"add-noise"` subtrace. This is a convenient way to select a subproblem and apply some proposal strategy to it.
- We call `make-constrained-generator` to create a version of `curve-model` that:
    - only samples at the addresses not already present in `fixed-choices` -- i.e., only samples a `noise-level` and `prob-outlier`;
    - returns not just a list of `ys`, but also a full trace of `curve-model` and a score.
- Our MCMC asymmetric proposers are responsible for returning a full trace of the model; we return the trace that the `constrained-generator` made for us.

This function is exactly the same as the old noise proposal, just implemented differently.

<a name="441"></a>
<h3 style="color:blue">Problem 4.4.1: Resimulation MH proposal

**Part 1: Rewrite curve-proposal**

In the same way as we did for the noise proposal, rewrite the curve proposal (which resamples the degree and coefficients of the underlying curve) to use a constrained generator:

In [ ]:
;; PROBLEM
(def new-curve-proposal
  (gen {:tracing-with t} [old-trace]
    (let [addresses-to-propose 
          '...
          
          [_ fixed-choices]
          '...
          
          constrained-generator
          '...
          
          [_ new-trace _]
          '...]
        new-trace)))

In [ ]:
;; SOLUTION
(def new-curve-proposal
  (gen {:tracing-with t} [old-trace]
    (let [addresses-to-propose 
          (filter #(address-contains? % "underlying-curve") (addresses-of old-trace))
          
          [_ fixed-choices]
          (partition-trace old-trace addresses-to-propose)
          
          constrained-generator
          (make-constrained-generator curve-model fixed-choices)
          
          [_ new-trace _]
          (t '() constrained-generator [xs])]
        new-trace)))

**Part 2: Write a general `make-resimulation-proposal` function**

Abstract away the details in the previous two functions to write a general `make-resimulation-proposal` function, which accepts a model, a set of inputs, and a predicate on addresses that selects which chocies to resimulate. It should return a resimulation proposer.

In [ ]:
;; PROBLEM

;; Fill in the ...
(def make-resimulation-proposal
  (gen [model inputs address-predicate]
    (gen {:tracing-with t} [old-trace]
      '...)))


;; So that the two proposals above can be defined as follows:
(def new-noise-proposer
  (make-resimulation-proposal curve-model [xs] #(address-contains? % "add-noise")))
(def new-curve-proposer
  (make-resimulation-proposal curve-model [xs] #(address-contains? % "underlying-curve")))

In [ ]:
;; SOLUTION
(def make-resimulation-proposal
  (gen [model inputs address-predicate]
    (gen {:tracing-with t} [old-trace]
      (let [addresses 
            (filter address-predicate (addresses-of old-trace))
            
            [_ fixed-choices] 
            (partition-trace old-trace addresses)
            
            constrained-generator 
            (make-constrained-generator model fixed-choices)
            
            [_ new-trace _] 
            (t '() constrained-generator inputs)]
          new-trace))))

;; So that the two proposals above can be defined as follows:
(def new-noise-proposer
  (make-resimulation-proposal curve-model [xs] #(address-contains? % "add-noise")))
(def new-curve-proposer
  (make-resimulation-proposal curve-model [xs] #(address-contains? % "underlying-curve")))

**Run inference with the new proposals**

If we run our inference using the above proposers, we should get the same results as before:

In [ ]:
(defn inference-step [xs ys tr]
  (let [model
        (with-args curve-model [xs])
        
        curve-step
        (custom-proposer-mh-step model new-curve-proposer)
        
        noise-step
        (custom-proposer-mh-step model new-noise-proposer)
        
        outlier-steps
        (map (fn [i] (symmetric-proposer-mh-step model (make-outlier-proposer i))) (range (count xs)))
        
        coeff-addresses
        (filter (fn [addr] (address-contains? addr "coeffs")) (addresses-of tr))
        
        coeffs-step
        (symmetric-proposer-mh-step model (make-gaussian-drift-proposer coeff-addresses 0.1))
        
        full-step
        (reduce comp identity `[~curve-step ~coeffs-step ~noise-step ~coeffs-step ~@outlier-steps])]
      
      (full-step tr)))

In [ ]:
(defn run-mh-curve-fitting
  [xs ys n-iters]
  (let [[_ initial-trace _]
        (infer-and-score :procedure curve-model :inputs [xs] :observation-trace (make-observation-trace ys))
                
        inferred-trace
        (reduce (fn [tr i] (inference-step xs ys tr))
            initial-trace (range n-iters))
        
        [_ _ score]
        (infer-and-score :procedure curve-model :inputs [xs] :observation-trace inferred-trace)]
      
      (pprint inferred-trace)
      (println (str "Final score: " score))))

In [ ]:
(run-mh-curve-fitting xs ys-quadratic 500)

**Because these proposals invoke `make-constrained-generator`, they will automatically use any special logic implemented by the generative function.**

# 5) Inference algorithms as generative functions

# 6) Modeling DSLs with specialized inference engines

HMM?
Multi-Mixture Model?
Logic programming?

# 7) Syntactic transformations

Autotracing? CPS?

# 8) Miniprob: implementing a featherweight probabilistic programming language

"Generative expressions" instead of generative functions.


In [ ]:
(defn make-generative-expression [run observe]
  {:run run, :observe observe})

(defn run [generative-expression] ((get generative-expression :run)))
(defn observe [generative-expression observations] ((get generative-expression :observe) observations))
(defn infer-and-score [generative-expression observations] (run (observe generative-expression observations)))

In [ ]:
(defn run [traceable] ((traceable :run)))
(defn generatet [traceable observations] ((traceable :generatet) observations))
(defn call-with-tracer [f]
  {:run #(f (fn [addr traceable] (run traceable))),
   :generatet
   (fn [observations]
     (call-with-tracer
       (fn [u]
         (let [score (atom 0), trace (atom {}),
               retval (f (fn [addr traceable]
                            (let [[v t s] (u addr (generatet traceable (maybe-subtrace observations addr)))]
                              (swap! score + s)
                              (swap! trace merge-subtrace addr t)
                              v)))]
             [retval (deref trace) (deref score)]))))})
(defn primitivet [sampler scorer]
  {:run sampler,
   :generatet
   (fn [observations]
     (call-with-tracer
      (fn [t]
        (let [result (if (empty? observations) (t '() (primitivet sampler scorer)) (trace-value observations))
              score  (if (empty? observations) 0 (scorer result))]
          [result {:value result} score]))))})
(defn from-custom-generatet [generatet]
  {:run #(first (run (generatet {}))), :generatet generatet})